<h1 style="font-size:3rem;color:orange;">Setting up</h1>

In [78]:
!git config --global user.email "jwinn19464@gmail.com"
!git config --global user.name "Jenifer Nguyen"

/bin/bash: /home/jenifer19464/miniconda3/envs/intel_gpu_env/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/jenifer19464/miniconda3/envs/intel_gpu_env/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [94]:
!git push -u origin main

/bin/bash: /home/jenifer19464/miniconda3/envs/intel_gpu_env/lib/libtinfo.so.6: no version information available (required by /bin/bash)
^C


In [ ]:
!ssh -T git@github.com

/bin/bash: /home/jenifer19464/miniconda3/envs/intel_gpu_env/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [92]:
!pwd

/bin/bash: /home/jenifer19464/miniconda3/envs/intel_gpu_env/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/home/jenifer19464/miniconda3


In [1]:
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import dgl.data
import torch
print(torch.__version__)
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
import pandas as pd
import numpy as np
import seaborn as sn 
from tqdm import tqdm

#RDKit packages 
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.rdBase import _vecti #_vectint on Windows
from rdkit import DataStructs
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
# from rdkit.Chem import PandasTools 
AllChem.SetPreferCoordGen(True)
from rdkit.Chem import rdmolops

from typing import Dict, List, Optional, Tuple
from tqdm import tqdm
from pathlib import Path

import csv
import matplotlib.pyplot as plt
# %matplotlib inline

# with one-hot
from dgllife.utils import (
    BaseAtomFeaturizer,
    BaseBondFeaturizer,
    atom_type_one_hot,
    bond_type_one_hot,
    smiles_to_bigraph,
)

from dgllife.data import Tox21, ClinTox, ToxCast
from dgllife.utils import SMILESToBigraph, CanonicalAtomFeaturizer, CanonicalBondFeaturizer
from dgllife.utils import mol_to_graph
from dgllife.utils import mol_to_bigraph

/home/jnguyen89/miniconda3/envs/thtn/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.2.2+cu121


<h1 style="font-size:3rem;color:orange;">Dataset</h1>

In [2]:
# load and save datasets for the first time

In [5]:
import os
import torch

# File paths
load_tox21 = './datasets/saved_tox21_dataset.pt'
load_clintox = './datasets/saved_clintox_dataset.pt'
load_toxcast = './datasets/saved_toxcast_dataset.pt'

# Initialize datasets
tox21 = None
clintox = None
toxcast = None

# Attempt to load datasets, or create them if not found
if os.path.exists(load_tox21):
    tox21 = torch.load(load_tox21)
    print(f"Dataset loaded from {load_tox21}")
else:
    print(f"{load_tox21} not found. Creating dataset...")
    
if os.path.exists(load_clintox):
    clintox = torch.load(load_clintox)
    print(f"Dataset loaded from {load_clintox}")
else:
    print(f"{load_clintox} not found. Creating dataset...")

if os.path.exists(load_toxcast):
    toxcast = torch.load(load_toxcast)
    print(f"Dataset loaded from {load_toxcast}")
else:
    print(f"{load_toxcast} not found. Creating dataset...")

# If any dataset is not loaded, create them
if tox21 is None or clintox is None or toxcast is None:
    # Convert SMILES notation into graphs with initial featurizations for nodes and edges
    smiles_to_g = SMILESToBigraph(node_featurizer=CanonicalAtomFeaturizer(), edge_featurizer=CanonicalBondFeaturizer())

    datasets = {
        'tox21': Tox21(smiles_to_g),
        'clintox': ClinTox(smiles_to_g),
        'toxcast': ToxCast(smiles_to_g)
    }

    for name, dataset in datasets.items():
        graphs = [graph for graph in dataset]
        save_path = f'./datasets/saved_{name}_dataset.pt'
        torch.save(dataset, save_path)
        print(f"Dataset created and saved to {save_path}")
    tox21 = torch.load(load_tox21)
    clintox = torch.load(load_clintox)
    toxcast = torch.load(load_toxcast)


Dataset loaded from ./datasets/saved_tox21_dataset.pt
Dataset loaded from ./datasets/saved_clintox_dataset.pt
Dataset loaded from ./datasets/saved_toxcast_dataset.pt


## Addressing the imbalance between positive and negative samples 

In [4]:
train_ids = torch.arange(1000)
tox21.task_pos_weights(train_ids)
train_ids = torch.arange(500)
clintox.task_pos_weights(train_ids)
toxcast.task_pos_weights(train_ids); # ; to suppress output 

AttributeError: 'NoneType' object has no attribute 'task_pos_weights'

<h1 style="font-size:3rem;color:orange">Tree Decomposition</h1>

## Download chemutils from GitHub then run the text file from terminal. 

In [5]:
#Link: mi_collections is a GitHub folder: https://github.com/Masatsugar/graph-decomposition-reassembling/blob/main/mi-collections/mi_collections/chemutils.py

## Tree Decomposition 

In [6]:
#Link: https://github.com/Masatsugar/graph-decomposition-reassembling/blob/main/mi-collections/mi_collections/moldr/moltree.py

from collections import defaultdict
from dgllife.utils.jtvae.chemutils import enum_assemble
from chemutils import get_clique_mol, get_mol, get_smiles, set_atommap
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree

MST_MAX_WEIGHT = 100
MAX_NCAND = 2000


def tree_decomp(mol):
    n_atoms = mol.GetNumAtoms()
    if n_atoms == 1:
        return [[0]], []

    cliques = []
    for bond in mol.GetBonds():
        a1 = bond.GetBeginAtom().GetIdx()
        a2 = bond.GetEndAtom().GetIdx()
        if not bond.IsInRing():
            cliques.append([a1, a2])

    ssr = [x for x in Chem.GetSymmSSSR(mol)]
    cliques.extend(ssr)

    nei_list = [[] for i in range(n_atoms)]
    for i in range(len(cliques)):
        for atom in cliques[i]:
            nei_list[atom].append(i)
    

# Merge Rings with intersection > 2 atoms
    for i in range(len(cliques)):
        if len(cliques[i]) <= 2:
            continue
        for atom in cliques[i]:
            for j in nei_list[atom]:
                if i >= j or len(cliques[j]) <= 2:
                    continue
                inter = set(cliques[i]) & set(cliques[j])
                if len(inter) > 2:
                    cliques[i].extend(cliques[j])
                    cliques[i] = list(set(cliques[i]))
                    cliques[j] = []

    cliques = [c for c in cliques if len(c) > 0]
    nei_list = [[] for i in range(n_atoms)]
    for i in range(len(cliques)):
        for atom in cliques[i]:
            nei_list[atom].append(i)

    # Build edges and add singleton cliques
    edges = defaultdict(int)
    for atom in range(n_atoms):
        if len(nei_list[atom]) <= 1:
            continue
        cnei = nei_list[atom]
        bonds = [c for c in cnei if len(cliques[c]) == 2]
        rings = [c for c in cnei if len(cliques[c]) > 4]
        # In general, if len(cnei) >= 3, a singleton should be added, but 1 bond + 2 ring is currently not dealt with.
        if len(bonds) > 2 or (len(bonds) == 2 and len(cnei) > 2):
            cliques.append([atom])
            c2 = len(cliques) - 1
            for c1 in cnei:
                edges[(c1, c2)] = 1
        elif len(rings) > 2:  # Multiple (n>2) complex rings
            cliques.append([atom])
            c2 = len(cliques) - 1
            for c1 in cnei:
                edges[(c1, c2)] = MST_MAX_WEIGHT - 1
        else:
            for i in range(len(cnei)):
                for j in range(i + 1, len(cnei)):
                    c1, c2 = cnei[i], cnei[j]
                    inter = set(cliques[c1]) & set(cliques[c2])
                    if edges[(c1, c2)] < len(inter):
                        edges[(c1, c2)] = len(
                            inter
                        )  # cnei[i] < cnei[j] by construction

    edges = [u + (MST_MAX_WEIGHT - v,) for u, v in edges.items()]
    if len(edges) == 0:
        return cliques, edges

    # Compute Maximum Spanning Tree
    row, col, data = (zip(*edges))
#     print(row)
#     print(col)
#     print(data)
    n_clique = len(cliques)
    clique_graph = csr_matrix((data, (row, col)), shape=(n_clique, n_clique))
    junc_tree = minimum_spanning_tree(clique_graph)
    row, col = junc_tree.nonzero()
    edges = [(row[i], col[i]) for i in range(len(row))]
    
# #     # Iterate through cliques and print atoms
#     for clique in cliques:
#         if isinstance(clique, list):
#             # If clique is a list, directly print its content
#             # print("Atoms in clique:", clique)
#         elif isinstance(clique, _vecti):
#             # If clique is an RDKit _vecti object, convert it to list and print
#             atom_indices = list(clique)
#             # print("Atoms in RDKit _vecti:", atom_indices)
#         # Access atoms in the molecule using atom_indices
#         # For example, if mol is your RDKit molecule object
#         # atoms = [mol.GetAtomWithIdx(atom_idx) for atom_idx in atom_indices]
#         # print("Atom objects in RDKit _vecti:", atoms)
#         else:
#             print("Unknown type found in cliques.")
            
    return (cliques, edges)
  


class MolTreeNode(object):
    def __init__(self, smiles, clique=[]):
        self.smiles = smiles
        self.mol = get_mol(self.smiles)
        if self.mol is None:
            print(f"MolTreeNode: Invalid SMILES: {smiles}")

        self.clique = [x for x in clique]  # create a copy of the 'clique' passed in 
        self.neighbors = []  #where neighboring MolTreeNode objects will be stored 

    def add_neighbor(self, nei_node):
        self.neighbors.append(nei_node)  #nei_node is a neighboring MolTreeNode object

    def recover(self, original_mol):
        clique = []
        clique.extend(self.clique)
        if not self.is_leaf:  #I don't see .is_leaf() defined anywhere. 
            for cidx in self.clique:
                original_mol.GetAtomWithIdx(cidx).SetAtomMapNum(self.nid)
                #This sets the atom map number of each atom in this clique (node) to the node's id. 
                #So, all atoms in the same clique will share the same id now. 

        for nei_node in self.neighbors:
            clique.extend(nei_node.clique)
            if nei_node.is_leaf:  # Leaf node, no need to mark
                continue
            for cidx in nei_node.clique:
                # allow singleton node override the atom mapping
                if cidx not in self.clique or len(nei_node.clique) == 1:
                    atom = original_mol.GetAtomWithIdx(cidx)
                    atom.SetAtomMapNum(nei_node.nid)

        clique = list(set(clique))
        label_mol = get_clique_mol(original_mol, clique)
        self.label = Chem.MolToSmiles(Chem.MolFromSmiles(get_smiles(label_mol)))

        for cidx in clique:
            original_mol.GetAtomWithIdx(cidx).SetAtomMapNum(0)

        return self.label

    def assemble(self):
        neighbors = [nei for nei in self.neighbors if nei.mol.GetNumAtoms() > 1]
        neighbors = sorted(neighbors, key=lambda x: x.mol.GetNumAtoms(), reverse=True)
        singletons = [nei for nei in self.neighbors if nei.mol.GetNumAtoms() == 1]
        neighbors = singletons + neighbors

        cands, aroma = enum_assemble(self, neighbors)
        new_cands = [cand for i, cand in enumerate(cands) if aroma[i] >= 0]
        if len(new_cands) > 0:
            cands = new_cands

        if len(cands) > 0:
            self.cands, _ = zip(*cands)
            self.cands = list(self.cands)
        else:
            self.cands = []


class MolTree(object):
    def __init__(self, smiles):
        self.smiles = smiles
        self.mol = get_mol(smiles)

        if self.mol is None:
            print(f"MolTree: Invalid SMILES: {smiles}")
            return

        # Stereo Generation (currently disabled)
        # mol = Chem.MolFromSmiles(smiles)
        # self.smiles3D = Chem.MolToSmiles(mol, isomericSmiles=True)
        # self.smiles2D = Chem.MolToSmiles(mol)
        # self.stereo_cands = decode_stereo(self.smiles2D)
        print('Open')
        cliques, edges = tree_decomp(self.mol)
        print('Close')
        self.nodes = []
        self.cliques = cliques
        self.edges = edges

        root = 0
        for i, c in enumerate(cliques):
            cmol = get_clique_mol(self.mol, c)
            node = MolTreeNode(get_smiles(cmol), c)
            self.nodes.append(node)
            if min(c) == 0:
                root = i

        for x, y in edges:
            self.nodes[x].add_neighbor(self.nodes[y])
            self.nodes[y].add_neighbor(self.nodes[x])

        if root > 0:
            self.nodes[0], self.nodes[root] = self.nodes[root], self.nodes[0]

        for i, node in enumerate(self.nodes):
            node.nid = i + 1
            if len(node.neighbors) > 1:  # Leaf node mol is not marked
                set_atommap(node.mol, node.nid)
            node.is_leaf = len(node.neighbors) == 1
            
def smiles2graph(smiles):
    # mol = Chem.MolFromSmiles(smiles)
    atom_featurizer = BaseAtomFeaturizer(
        {
            "n_feat": partial(
                atom_type_one_hot,
                allowable_set=["C", "N", "O", "F", "Si", "P", "S", "Cl" "Br", "I"],
                encode_unknown=True,
            )
        }
    )
    bond_featurizer = BaseBondFeaturizer({"e_feat": bond_type_one_hot})
    # bond_featurizer = BaseBondFeaturizer(
    #     {"e_feat": ConcatFeaturizer([bond_type_one_hot, bond_is_in_ring])}
    # )
    g = smiles_to_bigraph(
        smiles, node_featurizer=atom_featurizer, edge_featurizer=bond_featurizer
    )
    return g
    
    def size(self):
        return len(self.nodes)

    def recover(self):
        for node in self.nodes:
            node.recover(self.mol)

    def assemble(self):
        for node in self.nodes:
            node.assemble()

In [7]:
while True:
    choice = input('Which dataset do you want to use?\n 1 - Tox21\n 2 - ClinTox\n 3 - ToxCast\n')

    if choice == '1':
        dataset = tox21
        dataset_s = 'Tox21'
        print('Tox21 selected.')
        break
    elif choice == '2':
        dataset = clintox
        dataset_s = 'ClinTox'
        print('ClinTox selected.')
        break
    elif choice == '3':
        dataset = toxcast
        dataset_s = 'ToxCast'
        print('ToxCast selected.')
        break
    else:
        print('Invalid choice, please try again.')

Which dataset do you want to use?
 1 - Tox21
 2 - ClinTox
 3 - ToxCast
 1


Tox21 selected.


In [49]:
# Creating MolTree instances and appending to MolTree list
smiles_list = dataset[:][0]
graphs_list = dataset[:][1]
labels_list = dataset[:][2]
jt_list = []
data_list = list(zip(smiles_list, labels_list))

In [50]:
dataset[0]

('*C(=O)[C@H](CCCCNC(=O)OCCOC)NC(=O)OCCOC',
 Graph(num_nodes=24, num_edges=46,
       ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
       edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)}),
 tensor([1., 0.]),
 tensor([1., 1.]))

# Make dictionary to convert atom types and bond types into labels for node and edge features

In [75]:
atom_type = { 'C': 6,    # Carbon
    'N': 7,    # Nitrogen
    'O': 8,    # Oxygen
    'S': 16,   # Sulfur
    'F': 9,    # Fluorine
    'Si': 14,  # Silicon
    'P': 15,   # Phosphorus
    'Cl': 17,  # Chlorine
    'Br': 35,  # Bromine
    'Mg': 12,  # Magnesium
    'Na': 11,  # Sodium
    'Ca': 20,  # Calcium
    'Fe': 26,  # Iron
    'As': 33,  # Arsenic
    'Al': 13,  # Aluminum
    'I': 53,   # Iodine
    'B': 5,    # Boron
    'V': 23,   # Vanadium
    'K': 19,   # Potassium
    'Tl': 81,  # Thallium
    'Yb': 70,  # Ytterbium
    'Sb': 51,  # Antimony
    'Sn': 50,  # Tin
    'Ag': 47,  # Silver
    'Pd': 46,  # Palladium
    'Co': 27,  # Cobalt
    'Se': 34,  # Selenium
    'Ti': 22,  # Titanium
    'Zn': 30,  # Zinc
    'H': 1,
    'Li': 3,   # Lithium
    'Ge': 32,  # Germanium
    'Cu': 29,  # Copper
    'Au': 79,  # Gold
    'Ni': 28,  # Nickel
    'Cd': 48,  # Cadmium
    'In': 49,  # Indium
    'Mn': 25,  # Manganese
    'Zr': 40,  # Zirconium
    'Cr': 24,  # Chromium
    'Pt': 78,  # Platinum
    'Hg': 80,  # Mercury
    'Pb': 82   # Lead
}

bond_type = {'single': 1, 'double': 2, 'triple': 3, 'aromatic': 1.5}

In [76]:
len(atom_type)

43

# turn DGLGraph into gSpan format

In [52]:
# test with the first graph
g = graphs_list[0]
num_nodes = g.num_nodes()
src, dst = g.edges()
node_labels = g.ndata['h']
edge_labels = g.edata['e']  

In [77]:
smiles_list[0]

'*C(=O)[C@H](CCCCNC(=O)OCCOC)NC(=O)OCCOC'

In [53]:
g

Graph(num_nodes=24, num_edges=46,
      ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
      edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)})

# Create mappings using the feature vectors and the dictionaries of atom types and bond types

In [95]:
import dgl
import torch

def create_node_label(g):
    # Assuming `g` is your DGL graph and atom_type is your dictionary
    nfeats = g.ndata['h']  # Get the node features (one-hot encoded)
    
    # Create a reverse mapping from index to element symbol
    index_to_element = {index: element for index, element in enumerate(atom_type.keys())}

    # create list of atomic numbers to convert into a tensor of labels
    atomic_number_list = []
    
    # Iterate through each node in the graph
    for node_id in range(g.num_nodes()):
        # Get the one-hot encoded features for the current node
        one_hot_vector = nfeats[node_id][0:43]
    
        # Find the index of the one-hot encoded atom type (where the value is 1)
        atom_index = one_hot_vector.argmax().item()  # Get the index of the max value (1)
    
        # Use the index to get the corresponding element symbol
        element_symbol = index_to_element.get(atom_index)
    
        # Use the element symbol to get the atomic number
        atomic_number = atom_type.get(element_symbol)

        atomic_number_list.append(atomic_number)

    # convert to tensor
    labels = torch.tensor(atomic_number_list)
    
        # Print or store the result
        # print(f"Node ID: {node_id}, Element Symbol: {element_symbol}, Atomic Number: {atomic_number}")
    return labels
    
def create_edge_label(g):
    # Assuming `g` is your DGL graph and atom_type is your dictionary
    efeats = g.edata['e']  # Get the node features (one-hot encoded)
    
    # Create a reverse mapping from index to bond type
    index_to_bond = {index: bond for index, bond in enumerate(bond_type.keys())}
    bond_order_list = []
    
    # Iterate through each node in the graph
    for edge_id in range(g.num_edges()):
    
        one_hot_vector = efeats[edge_id][0:4]
        bond_index = one_hot_vector.argmax().item()  
        bond_label = index_to_bond.get(bond_index)
        bond_order = bond_type.get(bond_label)
        
        bond_order_list.append(bond_order)

    labels = torch.tensor(bond_order_list)
    
        # print(f"Edge ID: {edge_id}, Bond Type: {bond_label.upper()}, Bond Order: {bond_order}")
    return labels


In [46]:
sample = graphs_list[0:10]
sample

[Graph(num_nodes=16, num_edges=34,
       ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
       edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)}),
 Graph(num_nodes=15, num_edges=32,
       ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
       edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)}),
 Graph(num_nodes=21, num_edges=48,
       ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
       edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)}),
 Graph(num_nodes=20, num_edges=40,
       ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
       edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)}),
 Graph(num_nodes=11, num_edges=20,
       ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
       edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)}),
 Graph(num_nodes=20, num_edges=38,
       ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
       edata_schemes=

In [96]:
def convert_dgl_to_gspan(graphs_list, file_name):
    idx = 0
    with open(file_name, 'a') as f:
        for g in graphs_list:
            num_nodes = g.num_nodes()
            num_edges = g.num_edges()
            
            # Extract node and edge data
            src, dst = g.edges()

            # Check for node data and handle missing labels
            if 'h' in g.ndata:
                node_label = create_node_label(g)
            else:
                node_label = [-1] * num_nodes  # Assign -1 for all nodes if not found

            # Check for edge data and handle missing labels
            if 'e' in g.edata:
                edge_label = create_edge_label(g)
            else:
                edge_label = [-1] * num_edges  # Assign -1 for all edges if not found

            # Create gSpan format
            gspan_str = f"t # {idx}\n" 
            
            # Handle node labels
            for i in range(num_nodes):
                gspan_str += f"v {i} {int(node_label[i])}\n"

            # Handle edge labels
            for i in range(num_edges):
                gspan_str += f"e {src[i].item()} {dst[i].item()} {int(edge_label[i])}\n"

            f.write(gspan_str)
            idx += 1

convert_dgl_to_gspan(graphs_list, "graph_all_corrected.txt")

# end test section

In [10]:
len(smiles_list)

7831

In [11]:
len(dataset[0][2])

12

In [106]:
import logging
from collections import defaultdict
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree

# Set up logging
logging.basicConfig(filename='tree_decomp5.log', level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

MST_MAX_WEIGHT = 100

def tree_decomp(mol):
    n_atoms = mol.GetNumAtoms()
    if n_atoms == 0:
        logging.debug("No atoms in molecule.")
        return [], []

    if n_atoms == 1:
        return [[0]], []

    cliques = []
    for bond in mol.GetBonds():
        a1 = bond.GetBeginAtom().GetIdx()
        a2 = bond.GetEndAtom().GetIdx()
        if not bond.IsInRing():
            cliques.append([a1, a2])

    ssr = [x for x in Chem.GetSymmSSSR(mol)]
    cliques.extend(ssr)

    logging.debug(f"Initial cliques: {cliques}")

    nei_list = [[] for _ in range(n_atoms)]
    for i in range(len(cliques)):
        for atom in cliques[i]:
            nei_list[atom].append(i)

    logging.debug(f"Neighbor list: {nei_list}")

    # Merge Rings with intersection > 2 atoms
    for i in range(len(cliques)):
        if len(cliques[i]) <= 2:
            continue
        for atom in cliques[i]:
            for j in nei_list[atom]:
                if i >= j or len(cliques[j]) <= 2:
                    continue
                inter = set(cliques[i]) & set(cliques[j])
                if len(inter) > 2:
                    cliques[i].extend(cliques[j])
                    cliques[i] = list(set(cliques[i]))
                    cliques[j] = []

    logging.debug(f"Cliques: {cliques}")
    cliques = [c for c in cliques if len(c) > 0]
    logging.debug(f"Cliques after merging: {cliques}")
    
    nei_list = [[] for _ in range(n_atoms)]
    for i in range(len(cliques)):
        for atom in cliques[i]:
            nei_list[atom].append(i)

    # Build edges and add singleton cliques
    edges = defaultdict(int)
    for atom in range(n_atoms):
        if len(nei_list[atom]) <= 1:
            continue
        cnei = nei_list[atom]
        bonds = [c for c in cnei if len(cliques[c]) == 2]
        rings = [c for c in cnei if len(cliques[c]) > 4]

        if len(bonds) > 2 or (len(bonds) == 2 and len(cnei) > 2):
            cliques.append([atom])
            c2 = len(cliques) - 1
            for c1 in cnei:
                edges[(c1, c2)] = 1
        elif len(rings) > 2:
            cliques.append([atom])
            c2 = len(cliques) - 1
            for c1 in cnei:
                edges[(c1, c2)] = MST_MAX_WEIGHT - 1
        else:
            for i in range(len(cnei)):
                for j in range(i + 1, len(cnei)):
                    c1, c2 = cnei[i], cnei[j]
                    if c1 >= len(cliques) or c2 >= len(cliques):
                        logging.debug(f"Index out of range: c1={c1}, c2={c2}, cliques_length={len(cliques)}")
                        continue
                    inter = set(cliques[c1]) & set(cliques[c2])
                    if edges[(c1, c2)] < len(inter):
                        edges[(c1, c2)] = len(inter)

    edges = [u + (MST_MAX_WEIGHT - v,) for u, v in edges.items()]
    if len(edges) == 0:
        return cliques, edges

    # Compute Maximum Spanning Tree
    row, col, data = (zip(*edges))
    n_clique = len(cliques)
    clique_graph = csr_matrix((data, (row, col)), shape=(n_clique, n_clique))
    junc_tree = minimum_spanning_tree(clique_graph)
    row, col = junc_tree.nonzero()
    edges = [(row[i], col[i]) for i in range(len(row))]
    
    return (cliques, edges)


In [11]:
# create JT dataset

In [111]:
import logging
from tqdm import tqdm

# Set up logging
# logging.basicConfig(filename='tree_decomp2.log', level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# load_jt_dataset = f'./{dataset_s.lower()}_jt_dgl_dataset.pt'

# if os.path.exists(load_jt_dataset):
#     jt_dgl_dataset = torch.load(f'{dataset_s.lower()}_jt_dgl_dataset.pt')
#     print(jt_dgl_dataset)
# else:
#     jt_dgl_dataset = [(moltree_to_dglgraph(jt), label) for jt, label in tqdm(jt_list, desc='converting junction trees into dgl')]
#     torch.save(jt_dgl_dataset, f'{dataset_s.lower()}_jt_dgl_dataset.pt')  # Save the dataset to a file



problematic_smiles_file = 'out_of_index.txt'
jt_list = []

# Open the file to write problematic SMILES and their data
index = 0
with open(problematic_smiles_file, 'w') as f:
    for smiles, label in tqdm(data_list, desc="Processing SMILES"):
        index += 1
        try:
            mol_tree = MolTree(smiles)
            if mol_tree.mol is not None:
                jt_list.append((mol_tree, label))
        except Exception as e:
            # Log the problematic SMILES, label, and the error message
            error_message = f"Failed to process SMILES # {index}: {smiles} with error: {e}"
            logging.error(error_message)  # Log the error
            f.write(f"SMILES: {smiles}, Label: {label}, Error: {e}\n")  # Write detailed info to the file
            
# torch.save(jt_list, 'jt_dataset.pt')


Processing SMILES:   2%|▏         | 26/1480 [00:00<00:05, 246.30it/s]

Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open


Processing SMILES:   6%|▌         | 82/1480 [00:00<00:05, 262.13it/s]

Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open


Processing SMILES:   7%|▋         | 109/1480 [00:00<00:06, 203.14it/s]

...

Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open


Processing SMILES:  99%|█████████▊| 1458/1480 [00:12<00:00, 116.44it/s]

Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open


Processing SMILES: 100%|██████████| 1480/1480 [00:12<00:00, 121.09it/s]

Close
Open
Close
Open
Close
Open
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close


In [112]:
len(jt_list)

1454

In [118]:
type(jt_list[0][0])

__main__.MolTree

In [120]:
jt = jt_list[0][0]

In [121]:
dir(jt)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'cliques',
 'edges',
 'mol',
 'nodes',
 'smiles']

In [126]:
dir(jt.nodes[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'add_neighbor',
 'assemble',
 'clique',
 'is_leaf',
 'mol',
 'neighbors',
 'nid',
 'recover',
 'smiles']

In [130]:
jt.edges

[(0, 23),
 (1, 23),
 (2, 23),
 (2, 24),
 (3, 4),
 (3, 24),
 (4, 5),
 (5, 6),
 (6, 7),
 (7, 8),
 (8, 25),
 (9, 25),
 (10, 11),
 (10, 25),
 (11, 12),
 (12, 13),
 (13, 14),
 (15, 16),
 (15, 24),
 (16, 26),
 (17, 26),
 (18, 19),
 (18, 26),
 (19, 20),
 (20, 21),
 (21, 22)]

## Convert Junction Trees to DGLGraph

In [ ]:
from rdkit import Chem
from dgllife.utils import CanonicalAtomFeaturizer

In [132]:
node_featurizer = CanonicalAtomFeaturizer()
# with canonical
def aggregate_node_features(moltree):
    """Aggregate features for each node (clique) in the molecule tree."""
    node_features = []
    for node in moltree.nodes:
        # Use CanonicalAtomFeaturizer to featurize the node's molecule
        mol = node.mol
        node_feats = node_featurizer(mol)['h']
        node_feats = node_feats.mean(dim=0, keepdim=True)  # Average features across atoms in the clique
        node_features.append(node_feats)
    
    if len(node_features) == 0:
        print("Error: No node features were extracted.")
    
    return torch.cat(node_features) if node_features else torch.tensor([])

In [ ]:
# NOT COMPLETELY IMPLEMENTED YET!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

edge_featurizer = CanonicalBondFeaturizer()
# with canonical
def aggregate_edge_features(moltree):
    """Aggregate features for each edge in each clique in the molecule tree."""
    edge_features = []
    for edge in moltree.edges:
        # Use CanonicalBondFeaturizer to featurize edges
        mol = node.mol
        node_feats = node_featurizer(mol)['h']
        node_feats = node_feats.mean(dim=0, keepdim=True)  # Average features across atoms in the clique
        node_features.append(node_feats)
    
    if len(node_features) == 0:
        print("Error: No node features were extracted.")
    
    return torch.cat(node_features) if node_features else torch.tensor([])

In [135]:
from dgl import DGLGraph

def moltree_to_dglgraph(moltree):
    # Initialize a new DGLGraph
    g = DGLGraph()

    # Add nodes to the graph
    num_nodes = len(moltree.nodes)
    g.add_nodes(num_nodes)

    # Add edges to the graph
    src = []
    dst = []
    for edge in moltree.edges:
        src.append(edge[0])
        dst.append(edge[1])
    g.add_edges(src, dst)
    g.add_edges(dst, src)  # Add reverse edges since the graph is undirected

    # Assign node features    

    nfeats = aggregate_node_features(moltree)
    # if nfeats.shape[0] == 0:
    #     raise RuntimeError("No node features were generated.")
    # print(nfeats)
    g.ndata['h'] = nfeats


    return g

# load_jt_dataset = f'./{dataset_s.lower()}_jt_dgl_dataset.pt'

# if os.path.exists(load_jt_dataset):
#     jt_dgl_dataset = torch.load(f'{dataset_s.lower()}_jt_dgl_dataset.pt')
# else:
jt_dgl_dataset = [(moltree_to_dglgraph(jt), label) for jt, label in tqdm(jt_list, desc='converting junction trees into dgl')]
    # torch.save(jt_dgl_dataset, f'{dataset_s.lower()}_jt_dgl_dataset.pt')  # Save the dataset to a file

converting junction trees into dgl:   0%|          | 0/1454 [00:00<?, ?it/s]/home/jnguyen89/miniconda3/envs/thtn/lib/python3.12/site-packages/dgl/heterograph.py:92: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning(
converting junction trees into dgl: 100%|██████████| 1454/1454 [00:01<00:00, 771.15it/s]


In [136]:
len(jt_dgl_dataset)

1454

In [142]:
jt_g = jt_dgl_dataset[0][0]
jt_g

Graph(num_nodes=27, num_edges=52,
      ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
      edata_schemes={})

In [144]:
nfeats = jt_g.ndata['h']
len(nfeats)

27

In [ ]:
def convert_dgl_to_gspan(graphs_list, file_name):
    idx = 0
    with open(file_name, 'a') as f:
        for g in graphs_list:
            num_nodes = g.num_nodes()
            num_edges = g.num_edges()
            
            # Extract node and edge data
            src, dst = g.edges()

            # Check for node data and handle missing labels
            if 'h' in g.ndata:
                node_labels = g.ndata['h']
                scalar_node_labels = torch.sum(node_labels, dim=1).tolist()
            else:
                scalar_node_labels = [-1] * num_nodes  # Assign -1 for all nodes if not found

            # Check for edge data and handle missing labels
            if 'e' in g.edata:
                edge_labels = g.edata['e']
                scalar_edge_labels = torch.sum(edge_labels, dim=1).tolist()
            else:
                scalar_edge_labels = [-1] * num_edges  # Assign -1 for all edges if not found

            # Create gSpan format
            gspan_str = f"t # {idx}\n" 
            
            # Handle node labels
            for i in range(num_nodes):
                gspan_str += f"v {i} {int(scalar_node_labels[i])}\n"

            # Handle edge labels
            for i in range(num_edges):
                gspan_str += f"e {src[i].item()} {dst[i].item()} {int(scalar_edge_labels[i])}\n"

            f.write(gspan_str)
            idx += 1

convert_dgl_to_gspan(graphs_list, "graph_all.txt")

In [146]:
def dgl_to_gspan_input(g, label, graph_id):
    """
    Convert a DGLGraph `g` into gSpan's input format and return it as a string.
    """
    result = []
    result.append(f"t # {graph_id}")
    
    # Add nodes with their features (e.g., atom type)
    for node_id in range(g.num_nodes()):
        node_label = g.ndata['h'][node_id].argmax().item()  # Assuming one-hot encoded node features
        result.append(f"v {node_id} {node_label}")
    
    # Add edges (since the graph is undirected, avoid duplicates)
    edges = set()
    for src, dst in zip(*g.edges()):
        src, dst = src.item(), dst.item()
        if (src, dst) not in edges and (dst, src) not in edges:
            result.append(f"e {src} {dst} 1")  # Assume all edges have label 1
            edges.add((src, dst))

    return "\n".join(result)
def save_junction_trees_as_gspan_input(jt_list, output_file):
    """
    Convert a list of junction trees into gSpan input format and save to a file.
    """
    with open(output_file, 'w') as f:
        for graph_id, (moltree, label) in enumerate(jt_list):
            g = moltree_to_dglgraph(moltree)
            gspan_format = dgl_to_gspan_input(g, label, graph_id)
            f.write(gspan_format + "\n")

save_junction_trees_as_gspan_input(jt_list, 'jt_test.txt')

/home/jnguyen89/miniconda3/envs/thtn/lib/python3.12/site-packages/dgl/heterograph.py:92: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning(


In [ ]:
# reconstruct raw dataset

In [17]:
graphs_list = dataset[:][1]
raw_data_list = list(zip(graphs_list, labels_list))
dgl_dataset = [(graph, label) for graph, label in tqdm(raw_data_list, desc='creating raw dataset')]

creating raw dataset: 100%|██████████| 7831/7831 [00:00<00:00, 2770611.10it/s]


# Apply GNNs

In [18]:
import os
os.environ["DGLBACKEND"] = "pytorch"
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.data.utils import split_dataset
from torch.utils.data import DataLoader
from dgllife.data import Tox21
from dgllife.utils import smiles_to_bigraph, CanonicalAtomFeaturizer, CanonicalBondFeaturizer
from dgl.nn.pytorch import GraphConv, GATConv, GATv2Conv, SAGEConv, GINConv
from torch.nn import Linear, Sequential, BatchNorm1d, ReLU, Dropout
from rdkit import Chem
import numpy as np
import networkx as nx
from dgl.dataloading import GraphDataLoader
import concurrent.futures
import time
import torch.optim as optim

In [19]:
def collate(samples, dataset_type):
    graphs, labels = [], []
    i, j = 0, 1
    
    if dataset_type == 'Raw':
        i += 1
        j += 1
    for sample in samples:
        graphs.append(dgl.add_self_loop(sample[i]))  # Add self-loops during graph construction
        labels.append(sample[j])  # Extract the labels (3rd element is the labels tensor)
                    
    batched_graph = dgl.batch(graphs)
    labels = torch.stack(labels)
    return batched_graph, labels

# , edge_featurizer=edge_featurizer
while True:
    choice = input('Which dataset do you want to use?\n 1 - Raw\n 2 - Junction Tree\n')

    if choice == '1':
        graph_dataset = dgl_dataset
        dataset_type = 'Raw'
        print('Raw selected.')
        break
    elif choice == '2':
        graph_dataset = jt_dgl_dataset
        dataset_type = 'JT'
        print('JT selected.')
        break
    else:
        print('Invalid choice, please try again.')
        
train_set, val_set, test_set = split_dataset(graph_dataset, frac_list=[0.8, 0.1, 0.1], shuffle=True, random_state=12)

train_loader = DataLoader(train_set, batch_size=32, shuffle=True, collate_fn=lambda x: collate(x, dataset_type))
val_loader = DataLoader(val_set, batch_size=32, shuffle=False, collate_fn=lambda x: collate(x, dataset_type))
test_loader = DataLoader(test_set, batch_size=32, shuffle=False, collate_fn=lambda x: collate(x, dataset_type))

Which dataset do you want to use?
 1 - Raw
 2 - Junction Tree 2


JT selected.


In [20]:
len(list(train_loader))

194

In [21]:
from dgllife.utils import EarlyStopping, Meter

In [22]:
# Declare variables
hidden_size = 64
num_epochs = 100
checkpoint_dir = 'saveStates'
lr = 0.001

In [ ]:
# following code is redundant across the different GNNs, take user input to use raw or jt

In [23]:
# Check the first graph to get the feature dimension
first_graph = jt_dgl_dataset[0][0]
print(first_graph.ndata)
in_feats = first_graph.ndata['h'].shape[1]
print(in_feats)

# Determine the number of tasks
num_classes = dataset.labels.shape[1]
print(num_classes)

{'h': tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         1.0000, 0.0000],
        [0.5000, 0.0000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.00

In [24]:
def log_csv(dataset_s, model_name, hidden_features, epochs, learning_rate, test_accuracy, total_training_time, evaluation_time):
    # File path for logging
    file_path = 'training_log.csv'
    
    # Check if the file exists. If not, create it and add headers
    file_exists = os.path.isfile(file_path)
    
    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        
        # If the file doesn't exist, write the headers
        if not file_exists:
            writer.writerow(['Dataset', 'Model Name', 'Hidden Features', 'Epochs', 'Learning Rate', 'Test Accuracy', 'Training Time (s)', 'Evaluation Time (s)'])
        
        # Write the data
        writer.writerow([dataset_s, model_name, hidden_features, epochs, learning_rate, test_accuracy, total_training_time, evaluation_time])
    
    print(f"Training log saved to {file_path}")

# Run GCN

In [25]:
class GCN(nn.Module):
    def __init__(self, in_feats, hidden_size, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, hidden_size)
        self.conv2 = GraphConv(hidden_size, hidden_size)
        self.classify = nn.Linear(hidden_size, num_classes)

    def forward(self, g):
        h = g.ndata['h'].float()
        h = F.relu(self.conv1(g, h))
        h = self.conv2(g, h)
        with g.local_scope():
            g.ndata['h'] = h
            hg = dgl.mean_nodes(g, 'h')
        return self.classify(hg)

In [28]:
def train(model, dataloader, criterion, optimizer):
        model.train()
        train_meter = Meter()
        total_loss = 0
        for batched_graph, labels in dataloader:
    
            batched_graph = batched_graph.to(device)
            labels = labels.to(device)
            
            logits = model(batched_graph)
            loss = criterion(logits, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_meter.update(logits, labels)
            total_loss += loss.item()
    
            # Save checkpoint at the end of each epoch
            # save_checkpoint(model, optimizer, epoch, checkpoint_dir, 'saveState1.pth')
    
        return total_loss / len(dataloader)
    
def evaluate(model, dataloader):
    model.eval()
    eval_meter = Meter()
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for batched_graph, labels in dataloader:
            
            batched_graph = batched_graph.to(device)
            labels = labels.to(device)
            
            logits = model(batched_graph)
            preds = (logits > 0.5).float()
            eval_meter.update(preds, labels)
            total_correct += (preds == labels).sum().item()
            total_samples += labels.numel()
    return total_correct / total_samples

In [31]:
model_name = "GCN"

# Check the first graph to get the feature dimension
first_graph = jt_dgl_dataset[0][0]
in_feats = first_graph.ndata['h'].shape[1]
print(in_feats)

# Determine the number of tasks
num_classes = dataset.labels.shape[1]
print(num_classes)
model = GCN(in_feats, hidden_size, num_classes)

# Use DataParallel to wrap the model for parallel training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
best_val_acc = 0

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr)

for i in range(5):    
    print(f"Iteration {i+1}/5")
    start_time = time.time()
    
    # Train the model
    for epoch in tqdm(range(num_epochs), desc='Training progress'):
        loss = train(model, train_loader, criterion, optimizer)
        val_acc = evaluate(model, val_loader)
        if best_val_acc < val_acc:
                best_val_acc = val_acc
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss:.4f}, Validation Accuracy: {val_acc:.4f}, (best {best_val_acc:.3f})")
    
    end_time = time.time()
    training_time = end_time - start_time
    print(f"Training time: {training_time:.2f} seconds")
    
    # Measure the time for evaluation on the test set
    start_time = time.time()
    
    # Evaluate the model on the test set
    test_acc = evaluate(model, test_loader)
    print(f"Test Accuracy with {model_name}: {test_acc:.4f}")
    
    end_time = time.time()
    evaluation_time = end_time - start_time
    print(f"Evaluation time: {evaluation_time:.2f} seconds")
    
    log_csv(dataset_s, model_name, hidden_size, num_epochs, lr, test_acc, training_time, evaluation_time)

74
12


/home/jnguyen89/miniconda3/envs/thtn/lib/python3.12/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Iteration 1/5


Training progress:   1%|          | 1/100 [00:02<03:34,  2.17s/it]

Epoch 1/100, Loss: 0.2758, Validation Accuracy: 0.9390, (best 0.939)


...

Training progress:  99%|█████████▉| 99/100 [03:15<00:01,  1.99s/it]

Epoch 99/100, Loss: 0.1723, Validation Accuracy: 0.9410, (best 0.943)


Training progress: 100%|██████████| 100/100 [03:16<00:00,  1.97s/it]

Epoch 100/100, Loss: 0.1721, Validation Accuracy: 0.9413, (best 0.943)
Training time: 196.98 seconds


Test Accuracy with GCN: 0.9378
Evaluation time: 0.20 seconds
Training log saved to training_log.csv


In [ ]:
len(list(train_loader))

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
class dotdict(dict):
    def __getattr__(self, name):
        return self[name]

In [ ]:
# args = dotdict(
#     {
#         "layers": 32,
#         "optimizer": dotdict(
#             {
#                 "learning_rate": 0.01,
#                 "decay_steps": 10000,
#                 "decay_ratio": 0.5,
#                 "weight_decay": 5e-4,  # Weight for L2 loss
#             }
#         ),
#         "device": "cpu",
#         "metric": "acc",
#         "num_epochs": 10,
#         "patience": 50,
#     }
# )

# Run GAT

In [32]:
class GAT(nn.Module):
    def __init__(self, in_feats, hidden_size, num_classes):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_feats, hidden_size, num_heads=2)
        self.conv2 = GATConv(hidden_size * 2, hidden_size, num_heads=2)
        self.classify = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, g):
        h = g.ndata['h'].float()
        h = F.elu(self.conv1(g, h))
        h = h.view(h.size(0), -1)  # Flatten the output of conv1
        h = self.conv2(g, h)
        h = h.view(h.size(0), -1)  # Flatten the output of conv2
        with g.local_scope():
            g.ndata['h'] = h
            hg = dgl.mean_nodes(g, 'h')
        return self.classify(hg)

In [ ]:
 # def train(model, dataloader, criterion, optimizer):
 #        model.train()
 #        total_loss = 0
 #        for batched_graph, labels in dataloader:
    
 #            batched_graph = batched_graph.to(device)
 #            labels = labels.to(device)
            
 #            logits = model(batched_graph)
 #            loss = criterion(logits, labels)
 #            optimizer.zero_grad()
 #            loss.backward()
 #            optimizer.step()
 #            total_loss += loss.item()
 #        return total_loss / len(dataloader)
    
 #    def evaluate(model, dataloader):
 #        model.eval()
 #        total_correct = 0
 #        total_samples = 0
 #        with torch.no_grad():
 #            for batched_graph, labels in dataloader:
    
 #                batched_graph = batched_graph.to(device)
 #                labels = labels.to(device)
                
 #                logits = model(batched_graph)
 #                preds = (logits > 0.5).float()
 #                total_correct += (preds == labels).sum().item()
 #                total_samples += labels.numel()
 #        return total_correct / total_samples

In [33]:
model_name = "GAT"

# Check the first graph to get the feature dimension
first_graph = jt_dgl_dataset[0][0]
in_feats = first_graph.ndata['h'].shape[1]

# Determine the number of tasks
num_classes = dataset.labels.shape[1]
for i in range(5):
    print(f"Iteration {i+1}/5")
    model = GAT(in_feats, hidden_size, num_classes)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    best_val_acc = 0
    
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr)
    
    # Measure the time for training
    start_time = time.time()
    
    # Train the model
    for epoch in tqdm(range(num_epochs), desc='Training progress'):
        loss = train(model, train_loader, criterion, optimizer)
        val_acc = evaluate(model, val_loader)
        if best_val_acc < val_acc:
                best_val_acc = val_acc
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss:.4f}, Validation Accuracy: {val_acc:.4f}, (best {best_val_acc:.3f})")
    
    end_time = time.time()
    training_time = end_time - start_time
    print(f"Training time: {training_time:.2f} seconds")
    
    # Measure the time for evaluation on the test set
    start_time = time.time()
    
    # Evaluate the model on the test set
    test_acc = evaluate(model, test_loader)
    print(f"Test Accuracy with {model_name}: {test_acc:.4f}")
    
    end_time = time.time()
    evaluation_time = end_time - start_time
    print(f"Evaluation time: {evaluation_time:.2f} seconds")
    
    log_csv(dataset_s, model_name, hidden_size, num_epochs, lr, test_acc, training_time, evaluation_time)

Iteration 1/5


Training progress:   1%|          | 1/100 [00:02<03:34,  2.17s/it]

Epoch 1/100, Loss: 0.2437, Validation Accuracy: 0.9390, (best 0.939)


Training progress:   2%|▏         | 2/100 [00:04<03:33,  2.17s/it]

Epoch 2/100, Loss: 0.2120, Validation Accuracy: 0.9390, (best 0.939)


...

Training progress:  99%|█████████▉| 99/100 [03:36<00:02,  2.18s/it]

Epoch 99/100, Loss: 0.1821, Validation Accuracy: 0.9409, (best 0.942)


Training progress: 100%|██████████| 100/100 [03:38<00:00,  2.18s/it]

Epoch 100/100, Loss: 0.1824, Validation Accuracy: 0.9410, (best 0.942)
Training time: 218.36 seconds


Test Accuracy with GAT: 0.9393
Evaluation time: 0.21 seconds
Training log saved to training_log.csv


# Run GATv2

In [34]:
class GATv2(nn.Module):
    def __init__(self, in_feats, hidden_size, num_classes):
        super(GATv2, self).__init__()
        self.conv1 = GATv2Conv(in_feats, hidden_size, num_heads=2)
        self.conv2 = GATv2Conv(hidden_size * 2, hidden_size, num_heads=2)
        self.classify = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, g):
        h = g.ndata['h'].float()
        h = F.elu(self.conv1(g, h))
        h = h.view(h.size(0), -1)  # Flatten the output of conv1
        h = self.conv2(g, h)
        h = h.view(h.size(0), -1)  # Flatten the output of conv2
        with g.local_scope():
            g.ndata['h'] = h
            hg = dgl.mean_nodes(g, 'h')
        return self.classify(hg)

In [ ]:
# def train(model, dataloader, criterion, optimizer):
#         model.train()
#         total_loss = 0
#         for batched_graph, labels in dataloader:
    
#             batched_graph = batched_graph.to(device)
#             labels = labels.to(device)
            
#             logits = model(batched_graph)
#             # Debugging statement
#             loss = criterion(logits, labels)
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
#             total_loss += loss.item()
#         return total_loss / len(dataloader)
    
#     def evaluate(model, dataloader):
#         model.eval()
#         total_correct = 0
#         total_samples = 0
#         with torch.no_grad():
#             for batched_graph, labels in dataloader:
    
#                 batched_graph = batched_graph.to(device)
#                 labels = labels.to(device)
            
#                 logits = model(batched_graph)
#                 preds = (logits > 0.5).float()
#                 total_correct += (preds == labels).sum().item()
#                 total_samples += labels.numel()
#         return total_correct / total_samples

In [35]:
model_name = "GATv2"

# Check the first graph to get the feature dimension
first_graph = jt_dgl_dataset[0][0]
# print(first_graph.ndata)
in_feats = first_graph.ndata['h'].shape[1]
# hidden_size = 16

# Determine the number of tasks
num_classes = dataset.labels.shape[1]
for i in range(5):
    print(f"Iteration {i+1}/5")
    model = GATv2(in_feats, hidden_size, num_classes)
    
    # Use DataParallel to wrap the model for parallel training
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # model = nn.DataParallel(model)
    model.to(device)
    
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr)
    
    # Measure the time for training
    start_time = time.time()
    
    # Train the model
    for epoch in tqdm(range(num_epochs), desc='Training progress'):
        loss = train(model, train_loader, criterion, optimizer)
        val_acc = evaluate(model, val_loader)
        if best_val_acc < val_acc:
                best_val_acc = val_acc
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss:.4f}, Validation Accuracy: {val_acc:.4f}, (best {best_val_acc:.3f})")
    
    end_time = time.time()
    training_time = end_time - start_time
    print(f"Training time: {training_time:.2f} seconds")
    
    # Measure the time for evaluation on the test set
    start_time = time.time()
    
    # Evaluate the model on the test set
    test_acc = evaluate(model, test_loader)
    print(f"Test Accuracy with {model_name}: {test_acc:.4f}")
    
    end_time = time.time()
    evaluation_time = end_time - start_time
    print(f"Evaluation time: {evaluation_time:.2f} seconds")
    
    log_csv(dataset_s, model_name, hidden_size, num_epochs, lr, test_acc, training_time, evaluation_time)

Iteration 1/5


Training progress:   1%|          | 1/100 [00:02<03:29,  2.12s/it]

Epoch 1/100, Loss: 0.2453, Validation Accuracy: 0.9390, (best 0.942)


Training progress:   2%|▏         | 2/100 [00:04<03:34,  2.19s/it]

Epoch 2/100, Loss: 0.2119, Validation Accuracy: 0.9390, (best 0.942)


...

Training progress:  99%|█████████▉| 99/100 [03:35<00:02,  2.20s/it]

Epoch 99/100, Loss: 0.1687, Validation Accuracy: 0.9431, (best 0.945)


Training progress: 100%|██████████| 100/100 [03:37<00:00,  2.18s/it]

Epoch 100/100, Loss: 0.1682, Validation Accuracy: 0.9427, (best 0.945)
Training time: 217.78 seconds


Test Accuracy with GATv2: 0.9400
Evaluation time: 0.21 seconds
Training log saved to training_log.csv


In [23]:
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, hidden_size, num_classes, aggregator_type):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, hidden_size, aggregator_type)
        self.conv2 = SAGEConv(hidden_size, hidden_size, aggregator_type)
        self.classify = nn.Linear(hidden_size, num_classes)

    def forward(self, g):
        h = g.ndata['h'].float()
        h = F.elu(self.conv1(g, h))
        #h = h.view(h.size(0), -1)  # Flatten the output of conv1
        h = self.conv2(g, h)
        #h = h.view(h.size(0), -1)  # Flatten the output of conv2
        with g.local_scope():
            g.ndata['h'] = h
            hg = dgl.mean_nodes(g, 'h')
        return self.classify(hg)

In [24]:
# def train(model, dataloader, criterion, optimizer):
#     model.train()
#     total_loss = 0
#     for batched_graph, labels in dataloader:

#         batched_graph = batched_graph.to(device)
#         labels = labels.to(device)
        
#         logits = model(batched_graph)
#         # Debugging statement
#         loss = criterion(logits, labels)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#     return total_loss / len(dataloader)

# def evaluate(model, dataloader):
#     model.eval()
#     total_correct = 0
#     total_samples = 0
#     with torch.no_grad():
#         for batched_graph, labels in dataloader:

#             batched_graph = batched_graph.to(device)
#             labels = labels.to(device)
            
#             logits = model(batched_graph)
#             preds = (logits > 0.5).float()
#             total_correct += (preds == labels).sum().item()
#             total_samples += labels.numel()
#     return total_correct / total_samples

In [35]:
model_name = "GraphSAGE with mean aggregation"

# Check the first graph to get the feature dimension
first_graph = jt_dgl_dataset[0][0]
# print(first_graph.ndata)
in_feats = first_graph.ndata['h'].shape[1]
# hidden_size = 16
aggregator_type = 'mean'
# Determine the number of tasks
num_classes = dataset.labels.shape[1]

for i in range(5):
    print(f"Iteration {i+1}/5")
    model = GraphSAGE(in_feats, hidden_size, num_classes, aggregator_type)
    
    # Use DataParallel to wrap the model for parallel training
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # model = nn.DataParallel(model)
    model.to(device)
    best_val_acc = 0
    
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr)
    
    # Measure the time for training
    start_time = time.time()
    
    # Train the model
    for epoch in tqdm(range(num_epochs), desc='Training progress'):
        loss = train(model, train_loader, criterion, optimizer)
        val_acc = evaluate(model, val_loader)
        if best_val_acc < val_acc:
                best_val_acc = val_acc
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss:.4f}, Validation Accuracy: {val_acc:.4f}, (best {best_val_acc:.3f})")
    
    end_time = time.time()
    training_time = end_time - start_time
    print(f"Training time: {training_time:.2f} seconds")
    
    # Measure the time for evaluation on the test set
    start_time = time.time()
    
    # Evaluate the model on the test set
    test_acc = evaluate(model, test_loader)
    print(f"Test Accuracy with {model_name}: {test_acc:.4f}")
    
    end_time = time.time()
    evaluation_time = end_time - start_time
    print(f"Evaluation time: {evaluation_time:.2f} seconds")
    
    log_csv(dataset_s, model_name, hidden_size, num_epochs, lr, test_acc, training_time, evaluation_time)

Iteration 1/5


Training progress:   1%|          | 1/100 [00:02<04:09,  2.52s/it]

Epoch 1/100, Loss: 0.1428, Validation Accuracy: 0.9774, (best 0.977)


Training progress:   2%|▏         | 2/100 [00:04<03:54,  2.39s/it]

Epoch 2/100, Loss: 0.1022, Validation Accuracy: 0.9774, (best 0.977)


...

Training progress:  99%|█████████▉| 99/100 [03:41<00:02,  2.21s/it]

Epoch 99/100, Loss: 0.0930, Validation Accuracy: 0.9774, (best 0.977)


Training progress: 100%|██████████| 100/100 [03:43<00:00,  2.24s/it]

Epoch 100/100, Loss: 0.0931, Validation Accuracy: 0.9775, (best 0.977)
Training time: 223.77 seconds


Test Accuracy with GraphSAGE with mean aggregation: 0.9776
Evaluation time: 0.24 seconds
Training log saved to training_log.csv


In [36]:
model_name = "GraphSAGE with pool aggregation"

# Check the first graph to get the feature dimension
first_graph = jt_dgl_dataset[0][0]
# print(first_graph.ndata)
in_feats = first_graph.ndata['h'].shape[1]
# hidden_size = 16
aggregator_type = 'pool'

# Determine the number of tasks
num_classes = dataset.labels.shape[1]

for i in range(5):
    print(f"Iteration {i+1}/5")
    model = GraphSAGE(in_feats, hidden_size, num_classes, aggregator_type)
    
    # Use DataParallel to wrap the model for parallel training
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # model = nn.DataParallel(model)
    model.to(device)
    best_val_acc = 0
    
    
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr)

    # Measure the time for training
    start_time = time.time()
    
    # Train the model
    for epoch in tqdm(range(num_epochs), desc='Training progress'):
        loss = train(model, train_loader, criterion, optimizer)
        val_acc = evaluate(model, val_loader)
        if best_val_acc < val_acc:
                best_val_acc = val_acc
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss:.4f}, Validation Accuracy: {val_acc:.4f}, (best {best_val_acc:.3f})")
    
    end_time = time.time()
    training_time = end_time - start_time
    print(f"Training time: {training_time:.2f} seconds")
    
    # Measure the time for evaluation on the test set
    start_time = time.time()
    
    # Evaluate the model on the test set
    test_acc = evaluate(model, test_loader)
    print(f"Test Accuracy with {model_name}: {test_acc:.4f}")
    
    end_time = time.time()
    evaluation_time = end_time - start_time
    print(f"Evaluation time: {evaluation_time:.2f} seconds")
    
    log_csv(dataset_s, model_name, hidden_size, num_epochs, lr, test_acc, training_time, evaluation_time)

Iteration 1/5


Training progress:   1%|          | 1/100 [00:02<04:00,  2.43s/it]

Epoch 1/100, Loss: 0.1434, Validation Accuracy: 0.9774, (best 0.977)


Training progress:   2%|▏         | 2/100 [00:04<03:54,  2.40s/it]

Epoch 2/100, Loss: 0.1028, Validation Accuracy: 0.9774, (best 0.977)


...

Training progress:  99%|█████████▉| 99/100 [03:42<00:02,  2.21s/it]

Epoch 99/100, Loss: 0.0822, Validation Accuracy: 0.9775, (best 0.978)


Training progress: 100%|██████████| 100/100 [03:44<00:00,  2.25s/it]

Epoch 100/100, Loss: 0.0824, Validation Accuracy: 0.9776, (best 0.978)
Training time: 224.60 seconds


Test Accuracy with GraphSAGE with pool aggregation: 0.9776
Evaluation time: 0.23 seconds
Training log saved to training_log.csv


In [25]:
model_name = "GraphSAGE with lstm aggregation"

# Check the first graph to get the feature dimension
first_graph = jt_dgl_dataset[0][0]
# print(first_graph.ndata)
in_feats = first_graph.ndata['h'].shape[1]
# hidden_size = 16
aggregator_type = 'lstm'

# Determine the number of tasks
num_classes = dataset.labels.shape[1]

for i in range(5):
    print(f"Iteration {i+1}/5")
    model = GraphSAGE(in_feats, hidden_size, num_classes, aggregator_type)
    
    # Use DataParallel to wrap the model for parallel training
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # model = nn.DataParallel(model)
    model.to(device)
    best_val_acc = 0
    
    
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr)

    # Measure the time for training
    start_time = time.time()
    
    # Train the model
    for epoch in tqdm(range(num_epochs), desc='Training progress'):
        loss = train(model, train_loader, criterion, optimizer)
        val_acc = evaluate(model, val_loader)
        if best_val_acc < val_acc:
                best_val_acc = val_acc
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss:.4f}, Validation Accuracy: {val_acc:.4f}, (best {best_val_acc:.3f})")
    
    end_time = time.time()
    training_time = end_time - start_time
    print(f"Training time: {training_time:.2f} seconds")
    
    # Measure the time for evaluation on the test set
    start_time = time.time()
    
    # Evaluate the model on the test set
    test_acc = evaluate(model, test_loader)
    print(f"Test Accuracy with {model_name}: {test_acc:.4f}")
    
    end_time = time.time()
    evaluation_time = end_time - start_time
    print(f"Evaluation time: {evaluation_time:.2f} seconds")
    
    log_csv(dataset_s, model_name, hidden_size, num_epochs, lr, test_acc, training_time, evaluation_time)

Iteration 1/5


/home/jnguyen89/miniconda3/envs/thtn/lib/python3.12/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Training progress:   1%|          | 1/100 [00:04<07:28,  4.53s/it]

Epoch 1/100, Loss: 0.1499, Validation Accuracy: 0.9774, (best 0.977)


Training progress:   2%|▏         | 2/100 [00:08<07:03,  4.32s/it]

Epoch 2/100, Loss: 0.1027, Validation Accuracy: 0.9774, (best 0.977)


...

Training progress:  13%|█▎        | 13/100 [00:52<05:46,  3.98s/it]

Epoch 13/100, Loss: 0.0935, Validation Accuracy: 0.9774, (best 0.977)


Training progress:  14%|█▍        | 14/100 [00:56<05:42,  3.98s/it]

Epoch 14/100, Loss: 0.0933, Validation Accuracy: 0.9774, (best 0.977)


Training progress:  14%|█▍        | 14/100 [00:58<06:01,  4.21s/it]

KeyboardInterrupt



In [26]:
model_name = "GraphSAGE with gcn aggregation"

# Check the first graph to get the feature dimension
first_graph = jt_dgl_dataset[0][0]
# print(first_graph.ndata)
in_feats = first_graph.ndata['h'].shape[1]
# hidden_size = 16
aggregator_type = 'gcn'

# Determine the number of tasks
num_classes = dataset.labels.shape[1]
for i in range(5):
    print(f"Iteration {i+1}/5")
    model = GraphSAGE(in_feats, hidden_size, num_classes, aggregator_type)
    
    # Use DataParallel to wrap the model for parallel training
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # model = nn.DataParallel(model)
    model.to(device)
    best_val_acc = 0
    
    
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr)

    # Measure the time for training
    start_time = time.time()
    
    # Train the model
    for epoch in tqdm(range(num_epochs), desc='Training progress'):
        loss = train(model, train_loader, criterion, optimizer)
        val_acc = evaluate(model, val_loader)
        if best_val_acc < val_acc:
                best_val_acc = val_acc
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss:.4f}, Validation Accuracy: {val_acc:.4f}, (best {best_val_acc:.3f})")
    
    end_time = time.time()
    training_time = end_time - start_time
    print(f"Training time: {training_time:.2f} seconds")
    
    # Measure the time for evaluation on the test set
    start_time = time.time()
    
    # Evaluate the model on the test set
    test_acc = evaluate(model, test_loader)
    print(f"Test Accuracy with {model_name}: {test_acc:.4f}")
    
    end_time = time.time()
    evaluation_time = end_time - start_time
    print(f"Evaluation time: {evaluation_time:.2f} seconds")
    
    log_csv(dataset_s, model_name, hidden_size, num_epochs, lr, test_acc, training_time, evaluation_time)

Iteration 1/5


Training progress:   1%|          | 1/100 [00:02<03:51,  2.34s/it]

Epoch 1/100, Loss: 0.1663, Validation Accuracy: 0.9774, (best 0.977)


Training progress:   2%|▏         | 2/100 [00:04<03:46,  2.31s/it]

Epoch 2/100, Loss: 0.1025, Validation Accuracy: 0.9774, (best 0.977)


...

Epoch 98/100, Loss: 0.0958, Validation Accuracy: 0.9774, (best 0.977)


Training progress:  99%|█████████▉| 99/100 [03:35<00:02,  2.14s/it]

Epoch 99/100, Loss: 0.0958, Validation Accuracy: 0.9774, (best 0.977)


Training progress: 100%|██████████| 100/100 [03:37<00:00,  2.17s/it]

Epoch 100/100, Loss: 0.0959, Validation Accuracy: 0.9774, (best 0.977)
Training time: 217.31 seconds
Test Accuracy with GraphSAGE with gcn aggregation: 0.9777
Evaluation time: 0.20 seconds
Training log saved to training_log.csv


In [27]:
class GIN(nn.Module):
    def __init__(self, in_feats, hidden_size, num_classes):
        super(GIN, self).__init__()
        self.conv1 = GINConv(
            Sequential(Linear(in_feats, hidden_size),
                       BatchNorm1d(hidden_size), ReLU(),
                       Linear(hidden_size, hidden_size), ReLU()))
        self.conv2 = GINConv(
            Sequential(Linear(hidden_size, hidden_size), BatchNorm1d(hidden_size), ReLU(),
                       Linear(hidden_size, hidden_size), ReLU()))

        # self.conv3 = GINConv(
        #     Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
        #                Linear(dim_h, dim_h), ReLU()))
        
        #self.lin1 = Linear(dim_h*3, dim_h*3)
        self.lin1 = Linear(hidden_size*2, num_classes)
        self.classify = nn.Linear(hidden_size, num_classes)

    def forward(self, g):
        h = g.ndata['h'].float()
        h = F.relu(self.conv1(g, h))
        h = self.conv2(g, h)
        with g.local_scope():
            g.ndata['h'] = h
            hg = dgl.mean_nodes(g, 'h') # try sum_nodes
        return self.classify(hg)

In [ ]:
# def train(model, dataloader, criterion, optimizer):
#     model.train()
#     total_loss = 0
#     for batched_graph, labels in dataloader:

#         batched_graph = batched_graph.to(device)
#         labels = labels.to(device)
        
#         logits = model(batched_graph)
#         # Debugging statement
#         loss = criterion(logits, labels)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#     return total_loss / len(dataloader)

# def evaluate(model, dataloader):
#     model.eval()
#     total_correct = 0
#     total_samples = 0
#     with torch.no_grad():
#         for batched_graph, labels in dataloader:

#             batched_graph = batched_graph.to(device)
#             labels = labels.to(device)
            
#             logits = model(batched_graph)
#             preds = (logits > 0.5).float()
#             total_correct += (preds == labels).sum().item()
#             total_samples += labels.numel()
#     return total_correct / total_samples

In [ ]:
model_name = "GIN"

# Check the first graph to get the feature dimension
first_graph = jt_dgl_dataset[0][0]
# print(first_graph.ndata)
in_feats = first_graph.ndata['h'].shape[1]
# hidden_size = 16

# Determine the number of tasks
num_classes = dataset.labels.shape[1]
for i in range(5):
    print(f"Iteration {i+1}/5")
    model = GIN(in_feats, hidden_size, num_classes)
    
    # Use DataParallel to wrap the model for parallel training
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # model = nn.DataParallel(model)
    model.to(device)
    best_val_acc = 0
    
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr)
    # Measure the time for training
    start_time = time.time()
    
    # Train the model
    for epoch in tqdm(range(num_epochs), desc='Training progress'):
        loss = train(model, train_loader, criterion, optimizer)
        val_acc = evaluate(model, val_loader)
        if best_val_acc < val_acc:
                best_val_acc = val_acc
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss:.4f}, Validation Accuracy: {val_acc:.4f}, (best {best_val_acc:.3f})")
    
    end_time = time.time()
    training_time = end_time - start_time
    print(f"Training time: {training_time:.2f} seconds")
    
    # Measure the time for evaluation on the test set
    start_time = time.time()
    
    # Evaluate the model on the test set
    test_acc = evaluate(model, test_loader)
    print(f"Test Accuracy with {model_name}: {test_acc:.4f}")
    
    end_time = time.time()
    evaluation_time = end_time - start_time
    print(f"Evaluation time: {evaluation_time:.2f} seconds")
    
    log_csv(dataset_s, model_name, hidden_size, num_epochs, lr, test_acc, training_time, evaluation_time)


Iteration 1/5


Training progress:   1%|          | 1/100 [00:02<03:49,  2.31s/it]

Epoch 1/100, Loss: 0.1631, Validation Accuracy: 0.9774, (best 0.977)


Training progress:   2%|▏         | 2/100 [00:04<03:35,  2.20s/it]

Epoch 2/100, Loss: 0.0977, Validation Accuracy: 0.9774, (best 0.977)


...

Epoch 97/100, Loss: 0.0783, Validation Accuracy: 0.9774, (best 0.978)


Training progress:  98%|█████████▊| 98/100 [03:32<00:04,  2.17s/it]

Epoch 98/100, Loss: 0.0782, Validation Accuracy: 0.9775, (best 0.978)


Training progress:  99%|█████████▉| 99/100 [03:34<00:02,  2.16s/it]

Epoch 99/100, Loss: 0.0781, Validation Accuracy: 0.9771, (best 0.978)


In [ ]:
# list(jt_dgl_graph.nodes)

In [ ]:
tree_decomp(mol)

In [ ]:
# Creating a MolTree instance
smiles = dataset[:][0]
mol_tree = MolTree(smiles)

# Printing number of nodes
print("Number of nodes:", mol_tree.size())

# Recovering molecular structure
# mol_tree.recover()

# Assembling candidate structures
#mol_tree.assemble()

# Accessing individual nodes
for node in mol_tree.nodes:
    print("Node clique:", node.clique)


In [ ]:
dir(mol_tree)

In [ ]:
dir(dgl)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Create a new graph for junction tree
jtree = nx.Graph()

# Add nodes for each clique
for i, clique in enumerate(mol_tree.cliques):
    jtree.add_node(i, label=f"Clique {i}", atoms=clique)
    
# Add edges based on the separators (edges)
for x, y in mol_tree.edges:
    jtree.add_edge(x, y)

# Draw the junction tree
pos = nx.spring_layout(jtree)  # or use other layout algorithms such as circular_layout 
nx.draw(jtree, pos=pos, with_labels=True, node_color='lightblue', node_size=500, font_size=10)
labels = nx.get_edge_attributes(jtree, 'weight')  # if weights are available
nx.draw_networkx_edge_labels(jtree, pos, edge_labels=labels)

plt.title("Junction Tree")
plt.show()


In [ ]:
# code clips

In [ ]:
def load_problematic_smiles(filename):
    with open(filename, 'r') as file:
        smiles_list = file.read().splitlines()
    return smiles_list

# Load the problematic SMILES from the file into a variable
problematic_smiles = load_problematic_smiles('problematic_smiles.txt')

# Print the loaded SMILES strings
print("Loaded problematic SMILES:")
for smiles in problematic_smiles:
    print(smiles)


In [ ]:
def print_atom_info(mol, atoms):
    for atom_idx in atoms:
        atom = mol.GetAtomWithIdx(atom_idx)
        print(f"Atom index: {atom_idx}, Symbol: {atom.GetSymbol()}, Aromatic: {atom.GetIsAromatic()}, Degree: {atom.GetDegree()}")

print("Problematic clique atom details:")
print_atom_info(mol, [29, 21, 20, 6, 5, 4, 30])


In [ ]:
def print_bond_info(mol, atoms):
    for atom_idx in atoms:
        atom = mol.GetAtomWithIdx(atom_idx)
        for neighbor in atom.GetNeighbors():
            bond = mol.GetBondBetweenAtoms(atom.GetIdx(), neighbor.GetIdx())
            print(f"Bond between {atom.GetIdx()} ({atom.GetSymbol()}) and {neighbor.GetIdx()} ({neighbor.GetSymbol()}): Order = {bond.GetBondType()}, Aromatic = {bond.GetIsAromatic()}")

print("Problematic clique bond details:")
print_bond_info(mol, [29, 21, 20, 6, 5, 4, 30])


In [ ]:
from rdkit.Chem import Draw

def visualize_clique(mol, atoms):
    submol = Chem.PathToSubmol(mol, atoms)
    return Draw.MolToImage(submol)

image = visualize_clique(mol, [29, 21, 20, 6, 5, 4, 30])


In [ ]:
image

In [ ]:
dataset[0][0]

In [ ]:
print(dir(jt_list[0].nodes[0]))

for i in range((jt_list[0]).size()):
    print(jt_list[0].nodes[i].smiles)

# zip the L


In [ ]:
dataset[:][2]

<h1 style="font-size:3rem;color:orange">Preprocessing data for gSpan</h1>

In [ ]:
from typing import Dict, List, Optional, Tuple
from rdkit import Chem
from tqdm import tqdm

In [ ]:
# def create_junction_tree_dataset(mols):
# #     mols: List[Chem.rdchem.Mol],
# # ) -> Tuple[List[tuple], Dict[str, int]]:
#     """preprocess jucntion tree molecules to gSpan dataset.

#     :param mols:
#     :return:
#     """
#     tuple_list = []
#     cnt = 0
#     smiles2id = defaultdict()
#     for idx, mol in enumerate(tqdm(mols)):
#         smile = Chem.MolToSmiles(mol)
#         mt = MolTree(smile)
#         n_lab = defaultdict()
#         for i, node in enumerate(mt.nodes):
#             # node.smiles = node.smiles + str(":") + str(node.clique)
#             n_lab[i] = node.smiles
#             if node.smiles not in smiles2id:
#                 smiles2id[node.smiles] = cnt
#                 cnt += 1

#         # TODO: ADD EDGE LABEL, IONIZATION
#         edge_list = [1 for _ in range(len(mt.edges))]
#         node_list = [smiles2id[x.smiles] for x in mt.nodes]

#         s_id = ("t", "#", str(idx))
#         s_end = ("t", "#", "-1")
#         tuple_list.extend([s_id])
#         for i, n_label in enumerate(node_list):
#             # print("v", i, n_label)
#             tuple_list.append(("v", i, n_label))

#         edges_list = []
#         for (l, r), e_label in zip(mt.edges, edge_list):
#             # print("e", l, r, edgelabel)
#             edges_list.append(("e", l, r, e_label))

#         tuple_list.extend(edges_list)

#     tuple_list.extend([s_end])

#     return tuple_list, smiles2id


In [40]:
from rdkit import Chem
from collections import defaultdict
from tqdm import tqdm

def create_junction_tree_dataset(mols):
    """Preprocess junction tree molecules to gSpan dataset.

    :param mols: List[Chem.rdchem.Mol]
    :return: Tuple[List[tuple], Dict[str, int]]
    """
    tuple_list = []
    cnt = 0
    smiles2id = defaultdict()
    for idx, mol in enumerate(tqdm(mols)):
        if mol is None:
            print(f"Warning: Molecule at index {idx} is None")
            continue

        try:
            smile = Chem.MolToSmiles(mol)
            mt = MolTree(smile)
        except Exception as e:
            print(f"Error processing molecule at index {idx}: {e}")
            continue

        n_lab = defaultdict()
        for i, node in enumerate(mt.nodes):
            n_lab[i] = node.smiles
            if node.smiles not in smiles2id:
                smiles2id[node.smiles] = cnt
                cnt += 1

        # TODO: ADD EDGE LABEL, IONIZATION
        edge_list = [1 for _ in range(len(mt.edges))]
        node_list = [smiles2id[x.smiles] for x in mt.nodes]

        s_id = ("t", "#", str(idx))
        s_end = ("t", "#", "-1")
        tuple_list.extend([s_id])
        for i, n_label in enumerate(node_list):
            tuple_list.append(("v", i, n_label))

        edges_list = []
        for (l, r), e_label in zip(mt.edges, edge_list):
            edges_list.append(("e", l, r, e_label))

        tuple_list.extend(edges_list)

    tuple_list.extend([s_end])

    return tuple_list, smiles2id

# Ensure the dataset is not empty and all molecules are valid
mol_list = []
for i, smiles in enumerate(dataset):
    mol = Chem.MolFromSmiles(smiles[0])
    if mol is None:
        print(f"Warning: Invalid SMILES at index {i}: {smiles[0]}")
    mol_list.append(mol)

# Call the function with the prepared mol_list
create_junction_tree_dataset(mol_list)




  0%|                                                                        | 1/1480 [00:00<03:50,  6.42it/s]

Open
Close
Open
Close
Open
Close
Open
Close



  1%|▋                                                                      | 15/1480 [00:00<00:30, 47.62it/s]

Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close



  1%|▉                                                                      | 20/1480 [00:00<00:34, 42.40it/s]

...


 99%|████████████████████████████████████████████████████████████████████▍| 1469/1480 [01:03<00:00, 25.70it/s]

Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Error processing molecule at index 1472: list index out of range
Open


100%|█████████████████████████████████████████████████████████████████████| 1480/1480 [01:03<00:00, 23.43it/s]

Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close


([('t', '#', '0'),
  ('v', 0, 0),
  ('v', 1, 1),
  ('v', 2, 2),
  ('v', 3, 2),
  ('v', 4, 2),
  ('v', 5, 2),
  ('v', 6, 2),
  ('v', 7, 3),
  ('v', 8, 3),
  ('v', 9, 1),
  ('v', 10, 4),
  ('v', 11, 4),
  ('v', 12, 2),
  ('v', 13, 4),
  ('v', 14, 4),
  ('v', 15, 3),
  ('v', 16, 3),
  ('v', 17, 1),
  ('v', 18, 4),
  ('v', 19, 4),
  ('v', 20, 2),
  ('v', 21, 4),
  ('v', 22, 4),
  ('v', 23, 5),
  ('v', 24, 5),
  ('v', 25, 5),
  ('v', 26, 5),
  ('e', 0, 23, 1),
  ('e', 1, 23, 1),
  ('e', 2, 23, 1),
  ('e', 2, 24, 1),
  ('e', 3, 4, 1),
  ('e', 3, 24, 1),
  ('e', 4, 5, 1),
  ('e', 5, 6, 1),
  ('e', 6, 7, 1),
  ('e', 7, 8, 1),
  ('e', 8, 25, 1),
  ('e', 9, 25, 1),
  ('e', 10, 11, 1),
  ('e', 10, 25, 1),
  ('e', 11, 12, 1),
  ('e', 12, 13, 1),
  ('e', 13, 14, 1),
  ('e', 15, 16, 1),
  ('e', 15, 24, 1),
  ('e', 16, 26, 1),
  ('e', 17, 26, 1),
  ('e', 18, 19, 1),
  ('e', 18, 26, 1),
  ('e', 19, 20, 1),
  ('e', 20, 21, 1),
  ('e', 21, 22, 1),
  ('t', '#', '1'),
  ('v', 0, 6),
  ('v', 1, 6),
  ('v',

In [41]:
# turn entire dataset of smiles into RDKit mols
mol_list = []
for i in range(len(dataset)):
    mol = Chem.MolFromSmiles(dataset[i][0])
    mol_list.append(mol)

In [42]:
print(mol_list)

[<rdkit.Chem.rdchem.Mol object at 0x7f5d1caac0b0>, <rdkit.Chem.rdchem.Mol object at 0x7f5d1caacf90>, <rdkit.Chem.rdchem.Mol object at 0x7f5d1caad070>, <rdkit.Chem.rdchem.Mol object at 0x7f5d1caad0e0>, <rdkit.Chem.rdchem.Mol object at 0x7f5d1caad150>, <rdkit.Chem.rdchem.Mol object at 0x7f5d1caad1c0>, <rdkit.Chem.rdchem.Mol object at 0x7f5d1caad230>, <rdkit.Chem.rdchem.Mol object at 0x7f5d1caad2a0>, <rdkit.Chem.rdchem.Mol object at 0x7f5d1caad310>, <rdkit.Chem.rdchem.Mol object at 0x7f5d1caad380>, <rdkit.Chem.rdchem.Mol object at 0x7f5d1caad3f0>, <rdkit.Chem.rdchem.Mol object at 0x7f5d1caad460>, <rdkit.Chem.rdchem.Mol object at 0x7f5d1caad4d0>, <rdkit.Chem.rdchem.Mol object at 0x7f5d1caad540>, <rdkit.Chem.rdchem.Mol object at 0x7f5d1caad5b0>, <rdkit.Chem.rdchem.Mol object at 0x7f5d1caad620>, <rdkit.Chem.rdchem.Mol object at 0x7f5d1caad690>, <rdkit.Chem.rdchem.Mol object at 0x7f5d1caad700>, <rdkit.Chem.rdchem.Mol object at 0x7f5d1caad770>, <rdkit.Chem.rdchem.Mol object at 0x7f5d1caad7e0>,

In [43]:
from tqdm import tqdm
from collections import defaultdict
from rdkit import Chem

In [44]:
type(mol_list)

list

In [45]:
create_junction_tree_dataset(mol_list)


  0%|                                                                                | 0/1480 [00:00<?, ?it/s]

Open
Close
Open
Close
Open
Close
Open



  0%|▏                                                                       | 5/1480 [00:00<00:36, 40.84it/s]

Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open



  1%|▋                                                                      | 15/1480 [00:00<00:20, 71.35it/s]

...


 99%|████████████████████████████████████████████████████████████████████▍| 1469/1480 [01:02<00:00, 22.71it/s]

Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open



100%|█████████████████████████████████████████████████████████████████████| 1480/1480 [01:03<00:00, 23.46it/s]


Close
Open
Error processing molecule at index 1472: list index out of range
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close


([('t', '#', '0'),
  ('v', 0, 0),
  ('v', 1, 1),
  ('v', 2, 2),
  ('v', 3, 2),
  ('v', 4, 2),
  ('v', 5, 2),
  ('v', 6, 2),
  ('v', 7, 3),
  ('v', 8, 3),
  ('v', 9, 1),
  ('v', 10, 4),
  ('v', 11, 4),
  ('v', 12, 2),
  ('v', 13, 4),
  ('v', 14, 4),
  ('v', 15, 3),
  ('v', 16, 3),
  ('v', 17, 1),
  ('v', 18, 4),
  ('v', 19, 4),
  ('v', 20, 2),
  ('v', 21, 4),
  ('v', 22, 4),
  ('v', 23, 5),
  ('v', 24, 5),
  ('v', 25, 5),
  ('v', 26, 5),
  ('e', 0, 23, 1),
  ('e', 1, 23, 1),
  ('e', 2, 23, 1),
  ('e', 2, 24, 1),
  ('e', 3, 4, 1),
  ('e', 3, 24, 1),
  ('e', 4, 5, 1),
  ('e', 5, 6, 1),
  ('e', 6, 7, 1),
  ('e', 7, 8, 1),
  ('e', 8, 25, 1),
  ('e', 9, 25, 1),
  ('e', 10, 11, 1),
  ('e', 10, 25, 1),
  ('e', 11, 12, 1),
  ('e', 12, 13, 1),
  ('e', 13, 14, 1),
  ('e', 15, 16, 1),
  ('e', 15, 24, 1),
  ('e', 16, 26, 1),
  ('e', 17, 26, 1),
  ('e', 18, 19, 1),
  ('e', 18, 26, 1),
  ('e', 19, 20, 1),
  ('e', 20, 21, 1),
  ('e', 21, 22, 1),
  ('t', '#', '1'),
  ('v', 0, 6),
  ('v', 1, 6),
  ('v',

In [46]:
# redo and compare
create_junction_tree_dataset(mol_list)


  0%|▎                                                                       | 7/1480 [00:00<00:21, 69.28it/s]

Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close



  2%|█▏                                                                     | 24/1480 [00:00<00:26, 55.41it/s]

Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close



  3%|█▉                                                                     | 41/1480 [00:00<00:20, 68.55it/s]

...


 99%|████████████████████████████████████████████████████████████████████▏| 1463/1480 [01:03<00:00, 21.71it/s]

Close
Open
Close
Open
Close
Open
Close
Open
Close



100%|█████████████████████████████████████████████████████████████████████| 1480/1480 [01:03<00:00, 23.18it/s]


Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Error processing molecule at index 1472: list index out of range
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close


([('t', '#', '0'),
  ('v', 0, 0),
  ('v', 1, 1),
  ('v', 2, 2),
  ('v', 3, 2),
  ('v', 4, 2),
  ('v', 5, 2),
  ('v', 6, 2),
  ('v', 7, 3),
  ('v', 8, 3),
  ('v', 9, 1),
  ('v', 10, 4),
  ('v', 11, 4),
  ('v', 12, 2),
  ('v', 13, 4),
  ('v', 14, 4),
  ('v', 15, 3),
  ('v', 16, 3),
  ('v', 17, 1),
  ('v', 18, 4),
  ('v', 19, 4),
  ('v', 20, 2),
  ('v', 21, 4),
  ('v', 22, 4),
  ('v', 23, 5),
  ('v', 24, 5),
  ('v', 25, 5),
  ('v', 26, 5),
  ('e', 0, 23, 1),
  ('e', 1, 23, 1),
  ('e', 2, 23, 1),
  ('e', 2, 24, 1),
  ('e', 3, 4, 1),
  ('e', 3, 24, 1),
  ('e', 4, 5, 1),
  ('e', 5, 6, 1),
  ('e', 6, 7, 1),
  ('e', 7, 8, 1),
  ('e', 8, 25, 1),
  ('e', 9, 25, 1),
  ('e', 10, 11, 1),
  ('e', 10, 25, 1),
  ('e', 11, 12, 1),
  ('e', 12, 13, 1),
  ('e', 13, 14, 1),
  ('e', 15, 16, 1),
  ('e', 15, 24, 1),
  ('e', 16, 26, 1),
  ('e', 17, 26, 1),
  ('e', 18, 19, 1),
  ('e', 18, 26, 1),
  ('e', 19, 20, 1),
  ('e', 20, 21, 1),
  ('e', 21, 22, 1),
  ('t', '#', '1'),
  ('v', 0, 6),
  ('v', 1, 6),
  ('v',

In [47]:
def save_jt_dataset(mols, sname):
    """Save junction tree dataset for gSpan from Mols.

    :param mols:
    :param sname:
    :return:
    """
    tup_list, smiles2id = create_junction_tree_dataset(mols)
    id2smiles = {v: k for k, v in smiles2id.items()}
    
    # Convert sname to a Path object
    sname_path = Path(sname)
    
    # save gSpan data
    save_graph(tup_list, sname_path)

    # save vocabulary (node labels of cliques in JT) for reconstruction.
    pd.DataFrame.from_dict(id2smiles, orient="index", columns=["smiles"]).to_csv(
        f"{sname}vocab.csv"
    )

In [48]:
from pathlib import Path

In [49]:
def save_graph(tuple_list, fpath: Path) -> None:
    """save gSpan data.

    :param tuple_list:
    :param fpath:
    :return:
    """
    with fpath.open(mode="w") as f:
        for var_s in tuple_list:
            for var in var_s:
                f.write(str(var))
                f.write(" ")
            f.write("\n")
        print(f"save to {fpath}")

In [50]:
save_jt_dataset(mol_list, "ClinTox_JT.txt")


  0%|▏                                                                       | 5/1480 [00:00<00:39, 37.04it/s]

Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close



  1%|█                                                                      | 22/1480 [00:00<00:22, 64.57it/s]

Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close



  2%|█▍                                                                     | 29/1480 [00:00<00:29, 49.49it/s]

...

Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close



 99%|████████████████████████████████████████████████████████████████████▎| 1465/1480 [01:03<00:00, 27.40it/s]

Open
Close
Open
Close
Open
Close
Open
Close
Open
Close



100%|█████████████████████████████████████████████████████████████████████| 1480/1480 [01:03<00:00, 23.20it/s]


Open
Close
Open
Close
Open
Close
Open
Close
Open
Error processing molecule at index 1472: list index out of range
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
Open
Close
save to ClinTox_JT.txt


<h1 style="font-size:3rem;color:orange">gSpan</h1>

In [51]:
import collections
import os
import pickle
from collections import Counter
from dataclasses import dataclass
from pathlib import Path
from typing import List, Optional

import numpy as np
import pandas as pd
import rdkit.Chem as Chem
from matplotlib import pyplot as plt
from mi_collections.chemutils import (
    get_mol,
    get_smiles,
    mol_from_graph,
    mol_to_graph,
    mutag_convert,
    sanitize,
)
from mi_collections.ecfp.features import get_identifier
from mi_collections.gspan import gSpan
from mi_collections.gspan.config import parser
from mi_collections.gspan.main import main
from mi_collections.gspan.utils import (
    create_gspan_dataset_nx,
    gspan_to_mol,
    preprocess_mols,
)
from rdkit.Chem import Draw, Mol

ModuleNotFoundError: No module named 'mi_collections'

In [ ]:
def _report(self, projected):
        self._report_df = pd.DataFrame()
        self._frequent_subgraphs = [] 
        self.ggnx = []  

        self._frequent_subgraphs.append(copy.copy(self._DFScode))
        if self._DFScode.get_num_vertices() < self._min_num_vertices:
            return
        g = self._DFScode.to_graph(
            gid=next(self._counter), is_undirected=self._is_undirected
        )
        display_str = g.display()
        gx, vlbs, elbs = g.get_result()
        self.ggnx.append({"graph": gx, "vlbs": vlbs, "elbs": elbs, "g": g})
        # print('\nSupport: {}'.format(self._support))

        # Add some report info to pandas dataframe "self._report_df".
        ids = ",".join(map(str, self._ids))
        self._report_df = self._report_df.append(
            pd.DataFrame(
                {
                    "support": [self._support],
                    "description": [display_str],
                    "num_vert": self._DFScode.get_num_vertices(),
                    "support_ids": ids,
                },
                index=[int(repr(self._counter)[6:-1])],
            )
        )
        
         # Concatenate the current report data with the existing DataFrame
        self._report_df = pd.concat([self._report_df, report_data], ignore_index=True)
        
        if self._where:
            pass

In [ ]:
@dataclass
class DefaultConfig:
    data_path: Path
    support: int = 1
    lower: int = 2
    upper: int = 4
    directed: bool = False
    is_save: bool = True
    output_csv: bool = False
    method: str = "raw"


def check_substructures(path, max_nums=100, **kwargs):
    df = pd.read_csv(path, index_col=0)
    sub_mols = [Chem.MolFromSmiles(s) for s in df["0"]]
    Draw.MolsToGridImage(sub_mols[0:max_nums], **kwargs).show()


def get_matrix(report_df, wl_kernel, y):
    ncols = report_df.shape[0]
    nums = report_df.to_numpy()
    mat = np.zeros((len(y), ncols))
    for i in range(ncols):
        cnt = Counter(nums[i].split(","))
        for key, val in cnt.items():
            mat[int(key), i] = val

    mat = np.array(mat)
    X = np.c_[wl_kernel.X[0].X.toarray(), mat]
    return X


class GraphMining:
    def __init__(self, config: DefaultConfig):
        self.config = config
        self.save_name = None

    def _run_gspan(self):
        args_str = (
            f"-s {self.config.support} -d {self.config.directed} -l {self.config.lower} -u {self.config.upper} "
            f"-p False -w False {self.config.data_path}"
        )
        FLAG, _ = parser.parse_known_args(args=args_str.split())
        result = main(FLAG)
        return result

    def decompose(self, graphs):
        if any(graphs):
            create_gspan_dataset_nx(nx_graphs=graphs)
        gspan_object = self._run_gspan()
        return gspan_object


class MolsMining(GraphMining):
    def __init__(self, config: DefaultConfig):
        super(MolsMining, self).__init__(config=config)
        self.config = config
        self.save_name = None

    def decompose(self, mols: Optional[List[Mol]] = None) -> gSpan:
        if any(mols):
            preprocess_mols(
                mols, fname=self.config.data_path, method=self.config.method
            )

        gspan_object = self._run_gspan()
        if self.config.is_save:
            self.save_csv(gspan_object=gspan_object, mols=mols)

        return gspan_object

    def save_csv(
        self,
        gspan_object: gSpan,
        mols: Optional[List[Mol]] = None,
        suffix: str = ".pickle",
    ):
        cnf = self.config
        save_name = (
            f"{cnf.data_path.name.split('.')[0]}_s{cnf.support}l{cnf.lower}u{cnf.upper}"
        )

        self.save_name = save_name

        # Save as CSV
        smiles = [get_smiles(m) for m in mols]
        sub_mols = self.gspan_to_mols(gspan_object, smiles_list=smiles)
        sub_smiles = [get_smiles(m) for m in sub_mols]
        pd.DataFrame(sub_smiles).to_csv(
            cnf.data_path.with_name(save_name).with_suffix(".csv")
        )
        gspan_object._report_df["support_ids"].to_csv(
            cnf.data_path.with_name(f"{save_name}_info.csv")
        )

    def gspan_to_mols(self, gspan: gSpan, smiles_list: Optional[List[str]] = None):
        return gspan_to_mol(gspan, self.config.method, smiles_list=smiles_list)

    # @staticmethod
    # def save(fpath: Path, obj: gSpan):
    #     with fpath.open("wb") as f:
    #         pickle.dump(obj, f)

    @staticmethod
    def save(fpath: Path, obj: gSpan):
        # Ensure the directory exists
        if not fpath.parent.exists():
            fpath.parent.mkdir(parents=True, exist_ok=True)

        # Save the object
        with fpath.open("wb") as f:
            pickle.dump(obj, f)

    # @staticmethod
    # def load(fpath):
    #     with fpath.open("rb") as f:
    #         obj = pickle.load(f)
    #     return obj

    #  @staticmethod
    # def load(fpath):
    #     print(f"Loading gSpan object from {fpath}")
    #     with fpath.open("rb") as f:
    #         obj = pickle.load(f)
    #     return obj
    @staticmethod
    def load(fpath):
        print(f"Attempting to load from {fpath}")
        if not fpath.exists():
            # Check if a CSV version of the file exists
            csv_path = fpath.with_suffix('.csv')
            if csv_path.exists():
                print(f"Loading CSV version of the file from {csv_path}")
                return pd.read_csv(csv_path)
            else:
                raise FileNotFoundError(f"Neither {fpath} nor {csv_path} exist.")
        with fpath.open("rb") as f:
            obj = pickle.load(f)
        return obj

In [ ]:
if __name__ == "__main__":
    test_smiles = [
        "CC1CCC2=CC=CC=C2O1",
        "CC",
        "COC",
        "c1ccccc1",
        "CC1C(=O)NC(=O)S1",
        "CO",
    ]
    test_mols = [get_mol(s) for s in test_smiles]

    cnf = DefaultConfig(data_path=Path("outputs/tests/gspan_jt.data"), method="jt")
    # runner = MolsMining(config=cnf)
    # # gspan_obj = runner._run_gspan()
    # gspan_obj = runner.decompose(test_mols)
    # runner.save(Path("outputs/test/gspan_jt.pickle"), gspan_obj)
    # test = runner.load(cnf.data_path.with_name(runner.save_name))
    # runner = MolsMining(config=cnf)
    # gspan_obj = runner.decompose(test_mols)
    # save_path = Path("outputs/test/gspan_jt.pickle")
    # runner.save(save_path, gspan_obj)
    
    # load_path = cnf.data_path.with_name(runner.save_name)
    # print(f"Attempting to load from {load_path}")
    # test = runner.load(load_path)

    runner = MolsMining(config=cnf)
    gspan_obj = runner.decompose(test_mols)
    save_path = Path("outputs/test/gspan_jt.pickle")
    runner.save(save_path, gspan_obj)
    
    load_path = cnf.data_path.with_name(runner.save_name)
    print(f"Attempting to load from {load_path}")
    test = runner.load(load_path)

    sub_mols = gspan_to_mol(gspan_obj, method=cnf.method, smiles_list=test_smiles)
    sub_smiles = [get_smiles(m) for m in sub_mols]

    dat, info = get_identifier(test_mols[0], radius=2)
    dat = pd.DataFrame(dat).T.to_numpy().flatten()
    dat = [d for d in dat if d is not None]
    cd = collections.Counter(dat)
    ds = np.unique(dat)

In [ ]:
df = pd.DataFrame()

In [ ]:
dir(df)

In [ ]:
# !pip install --upgrade pandas


<h1 style="font-size:3rem;color:orange">Adjacency Matrix</h1>

## Unweighted adjacency matrix 

In [ ]:
# !pip install pysmiles

In [ ]:
# from pysmiles import read_smiles
# import networkx as nx
# import numpy as np 

# smiles = dataset[0][0]
# mol = read_smiles(smiles) 

# print(mol.nodes(data='element'))
# print(mol.edges(data='weight'))

# # Adjacency matrix
# adj_matrix = nx.to_numpy_array(mol)
# print(adj_matrix)

## Weighted adjacency matrix

In [ ]:
# from rdkit import Chem

# # Your RDKit mol object
# mol = Chem.MolFromSmiles(dataset[0][0])

# # Generate the molecule's adjacency matrix with bond order 
# G = Chem.GetAdjacencyMatrix(mol, useBO=True)

# # Convert the weighted adjacency matrix to a NumPy array
# adjacency_matrix = np.asarray(G)

# # Print the weighted adjacency matrix
# print(adjacency_matrix)

# # Convert weighted adjacency matrix to NetworkX graph
# G = nx.from_numpy_array(adjacency_matrix)
# nx.draw(G, with_labels=True)


In [ ]:
# import dgl

# # Assuming you have an existing DGLGraph `dgl_graph` with nodes and unweighted edges
# dgl_graph = dataset[0][1]
# # Example edge weights (replace this with your actual edge weights)
# edge_weights = G 

# # Add edge weights to DGLGraph
# dgl_graph.edata['weight'] = torch.tensor(edge_weights)  # Use torch.tensor for compatibility with DGL

# # Optionally, print or inspect edge weights
# print(dgl_graph.edata['weight'])


# Apply GCN for node classification

In [ ]:
# from dgl.nn import GraphConv
# class GCN(nn.Module):
#     #the __init__ method is the constructor for the class. 
#     #Whenever an object of this class is instantiated, the __init__ method is executed immediately, setting up the initial state of the object.
#     def __init__(self, in_feats, h_feats, num_classes): 
#         #This line calls the constructor of the parent class (nn.Module). 
#         #This is essential for ensuring that the underlying PyTorch functionalities tied to the nn.Module class are properly initialized for the GraphConv class.
#         super(GCN, self).__init__() 
#         self.conv1 = GraphConv(in_feats, h_feats)
#         self.conv2 = GraphConv(h_feats, num_classes)

#     def forward(self, g, in_feat):
#         h = self.conv1(g, in_feat)
#         h = F.relu(h)
#         h = self.conv2(g, h)
#         print(h.shape)
#         return h


# # Create the model with given dimensions
# model = GCN(h.ndata['h'].shape[1], 16, 2)

In [ ]:
# dataset[0][2][2]

In [ ]:
# def train(g, model):
#     optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
#     best_val_acc = 0
#     best_test_acc = 0

#     features = h.ndata['h']
#     labels = dataset[0][2][0]
#     #train_mask = g.ndata["train_mask"]
#     #val_mask = g.ndata["val_mask"]
#     #test_mask = g.ndata["test_mask"]
#     for e in range(100):
#         # Forward
#         logits = model(g, features)
#         print(logits)
        
#         '''
#         # Compute prediction
#         pred = logits.argmax(1)

#         # Compute loss
#         # Note that you should only compute the losses of the nodes in the training set.
#         loss = F.cross_entropy(logits[train_mask], labels[train_mask])

#         # Compute accuracy on training/validation/test
#         train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
#         val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
#         test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

#         # Save the best validation accuracy and the corresponding test accuracy.
#         if best_val_acc < val_acc:
#             best_val_acc = val_acc
#             best_test_acc = test_acc

#         # Backward
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         if e % 5 == 0:
#             print(
#                 f"In epoch {e}, loss: {loss:.3f}, val acc: {val_acc:.3f} (best {best_val_acc:.3f}), test acc: {test_acc:.3f} (best {best_test_acc:.3f})"
#             )


# model = GCN(g.ndata['atomic'].shape[1], 16, 2)
# train(g, model)
#         '''
        

In [ ]:
# model = GCN(h.ndata['h'].shape[1], 16, 2)
# train(g, model)

# Apply GCN for graph classification

In [ ]:
# dir(dataset)

In [ ]:
#print("Node feature dimensionality:", dataset.dim_nfeats)
#print("Number of graph categories:", dataset.gclasses)


from dgl.dataloading import GraphDataLoader

In [ ]:
from torch.utils.data.sampler import SubsetRandomSampler

# num_examples = len(dataset)
# num_train = int(num_examples * 0.8)
# num_val = int(num_examples * 0.1)
# num_test = num_examples - num_train - num_val

# train_sampler = SubsetRandomSampler(torch.arange(num_train))
# val_sampler = SubsetRandomSampler(torch.arange(num_train, num_train+num_val))
# test_sampler = SubsetRandomSampler(torch.arange(num_val, num_examples))

# train_dataloader = GraphDataLoader(
#     dataset, sampler=train_sampler, batch_size=32, drop_last=False
# )

# val_dataloader = GraphDataLoader(
#     dataset, sampler=val_sampler, batch_size=32, drop_last=False
# )

# test_dataloader = GraphDataLoader(
#     dataset, sampler=test_sampler, batch_size=32, drop_last=False
# )

In [ ]:
# it = iter(train_dataloader)
# batch = next(it)
# print(batch)

# #Number of batches 
# print(len(it))

# #Length of each batch 
# print(len(batch))

In [ ]:
# SMILES_string, batched_graph, labels, masks = batch
# print('graph: ', batched_graph)
# print(
#     "Number of nodes for each graph element in the batch:",
#     batched_graph.batch_num_nodes(),
# )
# print(
#     "Number of edges for each graph element in the batch:",
#     batched_graph.batch_num_edges(),
# )

# # Recover the original graph elements from the minibatch
# graphs = dgl.unbatch(batched_graph)

# print('graph: ', batched_graph)
# print("The original graphs in the minibatch:")
# print(graphs)

In [ ]:
# type(graphs)

In [ ]:
# type(batch)

In [ ]:
# batch[1]

In [ ]:
import dgl

# def add_self_loop_to_batch(batch):
#     # Unpack the batch tuple
#     SMILES_string, batched_graph, labels, masks = batch
    
#     # Extract each component graph
#     num_graphs = batched_graph.batch_size
#     graphs = dgl.unbatch(batched_graph)
    
#     # Add self-loops to all DGLGraphs in the batch
#     for i in range(num_graphs):
#         graphs[i] = dgl.add_self_loop(graphs[i])
    
#     # Recreate the batched graph from modified components
#     batched_graph = dgl.batch(graphs)
    
#     # Return the modified batch
#     return (SMILES_string, batched_graph, labels, masks)

In [ ]:
# print(type(batched_graph))
# print(batched_graph)
# #This means that batched_graph is just a big DGL graphs with sampled graphs as disconnected components. 

In [ ]:
# add_self_loop_to_batch(batch)

In [ ]:
from dgl.nn import GraphConv

# class GCN(nn.Module):
#     def __init__(self, in_feats, h_feats, num_classes):
#         super(GCN, self).__init__()
#         self.conv1 = GraphConv(in_feats, h_feats)
#         self.conv2 = GraphConv(h_feats, h_feats)
#         self.classify = nn.Linear(h_feats, num_classes)

# #     def forward(self, g, in_feat):
# #         h = self.conv1(g, in_feat)
# #         h = F.relu(h)
# #         h = self.conv2(g, h)
# #         g.ndata["h"] = h
# #         return dgl.mean_nodes(g, "h")

#     def forward(self, g):
#         h = g.ndata['h'].float()
#         h = F.relu(self.conv1(g, h))
#         h = self.conv2(g, h)
#         with g.local_scope():
#             g.ndata['h'] = h
#             hg = dgl.mean_nodes(g, 'h')
#         return self.classify(hg)


In [ ]:
# num_classes = dataset.labels.shape[1]
# print(num_classes)

In [ ]:
# list(train_dataloader)

In [ ]:
# # Create the model with given dimensions
# #model = GCN(in_feats, hidden_size, num_classes)
# model = GCN(74, 64, num_classes)   
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# #Training 
# num_epochs = 200

# criterion = nn.BCEWithLogitsLoss()

# for epoch in range(num_epochs):
#     for batch in train_dataloader:
#         batch = add_self_loop_to_batch(batch)
#         SMILES_string, batched_graph, labels, masks = batch
# #         print('labels: ', labels)
# #         print('g: ', batched_graph)
        
#         pred = model(batched_graph)
#         loss = criterion(pred, labels)   #F.cross_entropy(pred, labels)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
# #     print(loss)   

# # Validation loop
#     model.eval()
#     val_loss = 0.0
#     num_val_correct = 0
#     num_val_tests = 0

#     with torch.no_grad():
#         for batch in val_dataloader:
#             batch = add_self_loop_to_batch(batch)
#             SMILES_string, batched_graph, labels, masks = batch
#             logits = model(batched_graph)
#             val_loss += F.cross_entropy(logits, labels, reduction='sum').item()
            
#             preds = (logits > 0.5).float()
#             num_val_correct += (preds == labels).sum().item()
#             num_val_tests += labels.numel()

#     val_loss /= num_val_tests
#     val_accuracy = num_val_correct / num_val_tests
    
#     print(f"Epoch [{epoch + 1}/{num_epochs}], "
#           f"Train Loss: {loss.item():.4f}, "
#           f"Val Loss: {val_loss:.4f}, "
#           f"Val Accuracy: {val_accuracy:.4f}")

# # After training loop finishes, evaluate on test set
# model.eval()
# num_test_correct = 0
# num_test_tests = 0

# with torch.no_grad():
#     for batch in test_dataloader:
#         batch = add_self_loop_to_batch(batch)
#         SMILES_string, batched_graph, labels, masks = batch
#         logits = model(batched_graph)
#         preds = (logits > 0.5).float()
#         #preds = logits.argmax(dim=1)  #This generates a RuntimeError 
#         num_test_correct += (preds == labels).sum().item()
#         num_test_tests += labels.numel()

# test_accuracy = num_test_correct / num_test_tests
# print(f"Test Accuracy: {test_accuracy:.4f}")
    
# # num_correct = 0
# # num_tests = 0

# # for batch in test_dataloader:
# #     batch = add_self_loop_to_batch(batch)
# #     SMILES_string, batched_graph, labels, masks = batch
# #     logits = model(batched_graph, batched_graph.ndata['h'].float())
    
# #     #Thresholding logits using 0.5 to obtain binary predictions
# #     preds = (logits > 0.5).float()
# #     num_correct += (preds == labels).sum().item()
# #     num_tests += labels.numel()  # Can also use len(labels)

# # print("Test accuracy:", num_correct / num_tests)


In [ ]:
# print('pred size: ', preds.shape)
# print('labels size: ', labels.shape)